In [295]:
import hashlib
import hmac
import csv
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed
from multiprocessing import  Pool
import multiprocessing
import itertools
import random

In [296]:
from operator import itemgetter
import logging, sys

In [297]:
import warnings
warnings.filterwarnings("ignore")

In [298]:
_DEBUG = False
KAGGLE = False

In [299]:
# logging.basicConfig( #stream=sys.stdout, 
#                     level=logging.DEBUG)

In [300]:
# logging.debug('A debug message!')

In [301]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500
pd.set_option('display.max_colwidth', None)

In [302]:
l1=[0,1,2,3]
l2=['a','b','c','d']
l3=['hello','how','are','you']

ll=[l1,l2,l3]
# ll[0]=l1
# ll[1]=l2
# ll[2]=l3


list(zip(*ll))

[(0, 'a', 'hello'), (1, 'b', 'how'), (2, 'c', 'are'), (3, 'd', 'you')]

In [303]:
import time
from multiprocessing import Value
from ctypes import py_object

In [304]:
def init_pool_processes(shared_value):
    global p
    p = shared_value
def parallelize_dataframe(df, func, data_to_share=None,other_args=None, n_cores=4):
    start = time.time()
    df_split = np.array_split(df, n_cores)
    
    if data_to_share is not None:
        p = Value(py_object)
        p.value = data_to_share        
        pool = Pool(processes=n_cores,
                    initializer=init_pool_processes, initargs=(p,))
    else:
        pool = Pool(n_cores)
        
    arg_list=[]
    arg_list.append(df_split)
            
    if other_args is not None:
        for arg in other_args:
            arg_list.append(itertools.repeat(arg))
    
    parallel_args = zip(*arg_list)
    end = time.time()
    print(f'Parallel preparation time: {end-start}')
    df = pd.concat(pool.starmap(func, parallel_args))
    pool.close()
    pool.join()
    return df

In [305]:
def debugprint(msg):
    if _DEBUG:
        print(msg)

In [306]:
def get_bin(bins,val):
    matching_bin=-1
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            matching_bin= bins[i]
    if matching_bin==-1:
        matching_bin=bins[-1]
    return matching_bin

In [307]:
def calculate_roll(server_seed,client_seed,nonce):
    # Compute the HMAC-SHA512 hash of string1 using string2 as the key
    
    string1 = f"{nonce}:{server_seed}:{nonce}"
    hmac_key =   f"{nonce}:{client_seed}:{nonce}".encode()
    hmac_hash = hmac.new(key=hmac_key,
                         msg=string1.encode(),
                         digestmod=hashlib.sha512).hexdigest()

    # Convert the first 8 characters of the HMAC-SHA512 hash to an integer
    string3 = hmac_hash[:8]
    number = int(string3, 16)

    # Compute the roll value
    roll = round(number / 429496.7295)
    
    return roll

In [308]:
def compute_roll_hash_arrays(server_seed_array,client_seed_array,nonce_array):
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                           client_seed_array,
                                           nonce_array)
    
    # Compute roll for hash using the hash which is the next element in array
    # Vectorize the function
    vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
                                           client_seed_array[:chain_length-1],
                                           nonce_array[:chain_length-1])
    return roll_array,roll_array_hash

    

In [309]:
# trans_36 = bytes((x ^ 0x36) for x in range(256))
# trans_36

In [310]:
chain_length = 5110101 #1000001

filename = "sha256_hashchain_b9556671f785fe935bee087665b4047e421ea4491a5e2021a8152cab0b74c953.npy"

if KAGGLE:
    filepath_client ="/kaggle/input/hash-file-generation-client-seed/"
else:
    filepath_client="data/"
    
# filename_client=f'{filepath_client}sha256_hashchain_client_5M.npy'
filename_client=f'{filepath_client}sha256_hashchain_client1.npy'


# Define the number of CPU cores to use
num_cores = 8

In [311]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [312]:
# server_seed_array=np.load(filename)
# print(len(server_seed_array))
# server_seed_array[0:10]

In [313]:
client_seed_array=np.load(filename_client,
                         allow_pickle=True,fix_imports=True,encoding='latin1')
client_seed_array= client_seed_array[:chain_length]
print(len(client_seed_array))
client_seed_array[0:10]

1010102


array(['3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       '57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e',
       '3e49b34f2d6c7116d92ac8818c94f87d5616d7f2aea5024c2f38413605dc321c',
       'a0731d74c9501894d16ba144bc10dd49e8439c7f88304bfdbe4a755870c46da3',
       '665122519e5835c8e7ebfa0ab5b1949d75ac2c9e3dc1a6965a4b6e26bfc4ed76',
       '87fa02b24749184f0ce9183f21715ce556db72ecd1ad4f32b02227b40f3b24ab',
       'edee47aed4d9b598fa647145a17e44f9d155dc53ad231c913ff5c93fc04e56ee',
       'baf9d66be4136fe8acd30c042704b1798380fe0f3ee45d5800ae01db88e03ed3',
       '8eac9ebdcbfcf1720f6b0317fa66569c3129d278c0e2605069e7c5a0a2179c8d',
       'aeab33d31e0fb10105ce7ad75e3a40736db70a824c3ed6489957d6736243184b'],
      dtype='<U64')

In [314]:
# str_cat ='pppppp'
# # str_cat ='ppppppppppppppp'
# print(len(str_cat))
# client_seed_array_1=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_1[:2]

In [315]:
# str_cat ='ppppppppppppppp'
str_cat ='woxpwoxpwoxpwoxpwoxp'
print(len(str_cat))
client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
client_seed_array_2[:2]

20


array(['woxpwoxpwoxpwoxpwoxp3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       'woxpwoxpwoxpwoxpwoxp57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e'],
      dtype=object)

In [316]:
np.random.seed(5000)
low = 1500
# nonce_array = np.random.randint(low, high=low+chain_length, 
#                                 size=chain_length)

nonce_array = np.arange(low,low+chain_length,1)
np.random.shuffle(nonce_array)
print(len(nonce_array))
print(nonce_array[:10])
pd.Series(nonce_array).nunique()

5110101
[ 638661 1297273 1057802 1410000 2886373 4139846 2635511 4657935 2336816
  640280]


5110101

In [317]:
# roll_array,roll_array_hash=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)
# print(len(roll_array[0:10]))
# print(roll_array[0:100])
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [318]:
# roll_array_1,roll_array_hash_1=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_1,
#                                        nonce_array)
# print(len(roll_array_1[0:10]))
# print(roll_array_1[0:100])
# print(len(roll_array_hash_1[0:10]))
# print(roll_array_hash_1[0:100])

In [319]:
# roll_array_2,roll_array_hash_2=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_2,
#                                        nonce_array)
# print(len(roll_array_2[0:10]))
# print(roll_array_2[0:100])
# print(len(roll_array_hash_2[0:10]))
# print(roll_array_hash_2[0:100])

In [320]:
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)

# # Print the output
# print(len(roll_array[0:10]))
# print(roll_array[0:100])

In [321]:
# # Vectorize the function
# vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
#                                        client_seed_array[:chain_length-1],
#                                        nonce_array[:chain_length-1])

# # Print the output
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

#### Top k hash verification of client and server

In [322]:
# # server_hash_orig = "d100f357cf1b05cc23ddc35c5f34a960911f9e67582a6d5ff27d9a210137e9c0"
# # nonce= 2079
# # client_seed_orig = "ppppppd3f19c18078cea0c25643b203c9a56dc87cc5e71fa46c2cf5520ee8477f5b43c"
# # chain_length=100

# # server_hash_orig = "ffed0cf8eb7020128c153bea86b3f29bc328af0451052576211138023bb300bd"
# # nonce= 2078
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# # chain_length=100

# server_hash_orig = "68f1ed13e2f17e3d38fd888e470eaae28720675dc7c5913a6b7fd9586df159b4"
# nonce= 2116
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# client_seed_orig = "woxpwoxpwoxpwoxpwoxp"+"c023a89e8027c289bc936fb9a6be20aa5d1dcba191e70d98004e860fb6c0d5db"
# chain_length=1000



In [323]:
# Define a function to generate a single hash in the hash chain
def generate_hash(message):
    # Hash the message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Return the hash
    return hash_hex

In [324]:
def generate_hash_chain(seed,chain_length):
    hash_list=[seed]
    message = seed.encode()
    for i in range(chain_length):
        hash_hex= generate_hash(message)
        # Write the hash to the CSV file
        hash_list.append(hash_hex)
        # Update the message with the current hash
        message = hash_hex.encode()
    return hash_list

def compute_multirolls(hash_list,client_seed,nonce):
    rolls=[]
    for cur_hash in hash_list:
        roll = calculate_roll(cur_hash,client_seed,nonce)
        rolls.append(roll)
    return rolls
def compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce):
    rolls=[]
    for cur_hash_nonce in hash_list_nonce:
        roll = calculate_roll(server_hash,client_seed,cur_hash_nonce)
        rolls.append(roll)
    return rolls

Function description 

Input
--------
Server Hash
Nonce
Client Seed

Output
----------
Category that has more number of successes - GT or LT <br>
How many number of success ?<br>

Return state of each condition: (Pass, Fail, Tie) <br>
100 values
-----------

1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin

1000 values
----------------
1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin


In [325]:
ABOVE="Above"
BELOW="Below"
INTER ="Intermediate"
def set_tristate(val1,val2):
    if val1 == val2:
        below_state=-1
        above_state =-1
    else:
        below_state = val1 > val2
        above_state = int(~below_state)
        below_state = int(below_state)
    return below_state,above_state

# def set_diffstate(val1,val2):
#     below_state = val1-val2
#     above_state = -1 * below_state
#     return below_state,above_state
def compute_least_condition(bin_counts):
    mask = bin_counts.index < 5
    val_below = bin_counts[mask].min()
    val_above = bin_counts[~mask].min()
    #NOTE: here val_above and val_below are swapped in arguments
    #since least value in below state should be lesser than above state
    below_state,above_state=set_tristate(val_above,val_below)
    return below_state,above_state
def compute_mostlow_condition(bin_counts,low_values_cutoff):
    mask = bin_counts < low_values_cutoff
    mask2 = bin_counts.index < 5
    val_below = bin_counts[mask & mask2].count()
    val_above = bin_counts[mask & ~mask2].count()
    
#     print(f'val_below:{val_below} val_above:{val_above}')
    below_state,above_state=set_tristate(val_below,val_above)
    return below_state,above_state

def compute_conditions(rolls_arr,size,initial_value=None):
    
    rolls_arr = rolls_arr[:size]
    low_values_cutoff = size * 0.1
    
    below_conditions = np.full(5,-100)
    above_conditions = np.full(5,-100)
    #Condition 1: 
    #Total number of low values on the given category is higher than that of other category 
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts_cond1 = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Sum bincounts')
    debugprint(bin_counts_cond1)
#     print(bin_counts.loc[0],bin_counts.loc[1])
    
    below_conditions[0],above_conditions[0]=set_tristate(bin_counts_cond1.loc[1],bin_counts_cond1.loc[0])
    
    bins = list(range(0,10001,1000))
    labels = list(range(0,10))
    bin_counts = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Full bincounts')
    debugprint(bin_counts)
    #Condition 2: 
    #This category has the least value
    below_conditions[1],above_conditions[1]=compute_least_condition(bin_counts)
    
    #Condition 3: 
    #This category has the most number of low values
    below_conditions[2],above_conditions[2]=compute_mostlow_condition(bin_counts,low_values_cutoff)
    
#     print(f'INITIAL roll:  {rolls_arr[0]}')g
    if initial_value is None:
        initial_value=rolls_arr[0]
    initial_bin = get_bin(labels,int(initial_value/1000))
    
    if initial_bin<5:
        mask_filt = bin_counts.index > initial_bin
    else:
        mask_filt = bin_counts.index < initial_bin
    debugprint(f'filtered bincounts with initial bin:{initial_bin}')
    debugprint(bin_counts[mask_filt])
    #Condition 4: 
    #This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
    below_conditions[3],above_conditions[3]=compute_least_condition(bin_counts[mask_filt])
    
    #Condition 5: 
    #This category has the most number of low values
    below_conditions[4],above_conditions[4]=compute_mostlow_condition(bin_counts[mask_filt],
                                                                      low_values_cutoff)
   
    return below_conditions,above_conditions,bin_counts_cond1,bin_counts
    
def predict_seed_output(server_hash,nonce,client_seed,
                        hash_list_server=None,hash_list_nonce=None,
                        initial_value=None):
    chain_length=2000
    if (hash_list_server is None) and (hash_list_nonce is None):
        hash_list_server = generate_hash_chain(server_hash,chain_length)
#     print('Hash chain completed')
    if hash_list_nonce is not None:
        rolls_list  = compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce)
    else:
        rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
#     print('Roll computation completed')
    #100 values conditions
    rolls_arr=np.array(rolls_list)
    cond_100_below,cond_100_above,\
    bin_counts_cond1_100,bin_counts_100 = compute_conditions(rolls_arr,100,initial_value)
    cond_1000_below,cond_1000_above,\
    bin_counts_cond1_1000,bin_counts_1000 = compute_conditions(rolls_arr,1000,initial_value)
    cond_2000_below,cond_2000_above,\
    bin_counts_cond1_2000,bin_counts_2000 = compute_conditions(rolls_arr,2000,initial_value)
#     cond_4000_below,cond_4000_above = compute_conditions(rolls_arr,4000)
    
    cond_below_all = np.concatenate((cond_100_below,cond_1000_below,
                                     cond_2000_below,
#                                      cond_4000_below
                                    ))
    cond_above_all = np.concatenate((cond_100_above,cond_1000_above,
                                     cond_2000_above,
#                                      cond_4000_above
                                    ))
    
    debugprint(cond_below_all)
    debugprint(cond_above_all)
    
    below_passes = (cond_below_all == 1).sum()
    above_passes = (cond_above_all == 1).sum()
    
    if below_passes > above_passes:
        cond_all =  cond_below_all
        passed_state = BELOW
        total_passes = below_passes
    else:
        cond_all = cond_above_all
        passed_state = ABOVE
        total_passes = above_passes
  
    diff = abs(below_passes-above_passes)
    bin_counts_list = [bin_counts_cond1_100,bin_counts_100,
                      bin_counts_cond1_1000,bin_counts_1000,
                      bin_counts_cond1_2000,bin_counts_2000]
    
    if (bin_counts_cond1_100.loc[-1] < 5) & \
    (bin_counts_cond1_1000.loc[-1] < 50) & (bin_counts_cond1_2000.loc[-1] < 100):
        passed_state = INTER
    

    return cond_all,diff,total_passes,passed_state,bin_counts_list



In [326]:
def compare_prev(server_seed_array,
                client_seed,
                nonce_array,value_to_match=None):
#     print('compare_prev STARTED')
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                       client_seed,
                                       nonce_array)
    if value_to_match is None:
        value_to_match = roll_array[0]
    indices =np.where(roll_array==value_to_match)[0]
    indices_prev = indices -1 
#     print(len(indices_prev))
#     print(np.sort(roll_array[indices_prev]))
    prev_roll_match = roll_array[indices_prev]
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts = pd.cut(roll_array, bins,labels=labels).value_counts()
#     print('Sum bincounts')
#     print(bin_counts)
    
    return bin_counts

def validate_test_topk(test,client_seed_data,pass_exp1,pass_exp2,
                       pass_state_reqd=None,last_index=None,result_df=None,
                       client_char_count=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    bincountcols=[]
    bincountprevcols=[]
    sumbincols = ['lt4750','inter','gt5250']
    
    for col in sumbincols:
        bincountprevcols.append('prev_'+str(col))
    bincountprevcols_nonce =['nonce_'+col for col in bincountprevcols]
    for hash_size in [100,1000,2000]:
        for col in sumbincols:
            bincountcols.append(str(hash_size)+'_'+str(col))
        for col in range(10):
            bincountcols.append(str(hash_size)+'_'+str(col))
#     print(bincountcols)

    bincountcols_nonce = ['nonce_'+col for col in bincountcols]
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'total_passes','passed_state',
                                    'total_passes_1','total_passes_2',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] + bincountprevcols+bincountprevcols_nonce +\
                                             bincountcols+bincountcols_nonce)
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
#         if server_count % 10==0:
#             i = len(client_seed_data)-1
#             print(f'server_count:{server_count}')
        
#         if not result_success:
#             i= last_success_index
#             i = len(client_seed_data)-1
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        chain_length=2000
        cur_hash_list = generate_hash_chain(cur_hash,chain_length)
        cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
#         cur_hash_list_rev = generate_hash_chain(cur_hash[::-1],chain_length)
        
#         client_seed_data=np.array(cur_hash_list[1:])
#         client_seed_data=(str(cur_nonce) + pd.Series(client_seed_data)).values
        
##         cur_hash_list=list((cur_hash + pd.Series(np.array(cur_hash_list))).values)
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 1000==0:
                print(f'current scan : {i}')
            
            if client_char_count is None:
                client_seed = client_seed_data[i]
            else:
                client_seed = client_seed_data[i][:client_char_count]

#             client_seed= generate_hash((client_seed_data[i] + prev_match).encode())
#             client_seed= generate_hash((str(cur_nonce)+client_seed_data[i]).encode())
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            if (3000<=roll_hash<=7000) or ((i+1) in last_indices):
                total_passes=-1
                passed_state=ABOVE
                passed_state_2=BELOW
 
            else:
                cond_all,diff,total_passes_1,passed_state,bin_counts_list = predict_seed_output(cur_hash,
                                                                              cur_nonce,
                                                                              client_seed,
                                                                             hash_list_server=cur_hash_list)
                #This condition is to reduce time execution
                total_passes_2=0
                passed_state_2=passed_state
                if (pass_exp2>=total_passes_1):
                    cond_all_2,diff_2,total_passes_2,passed_state_2,bin_counts_list_nonce = predict_seed_output(cur_hash,
                                                                                  cur_nonce,
                                                                                  client_seed,
                                                                                 hash_list_nonce=cur_hash_list_nonce)
#                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_nonce_2 = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              hash_list_nonce=cur_hash_list_nonce)
# #                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_rev = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              cur_hash_list_rev)
                total_passes = total_passes_1 + total_passes_2  

            i-=1
#             if (total_passes_1>=pass_exp1) & (total_passes_2>=pass_exp2) & (passed_state==passed_state_2):
#             if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
#                 & (last_index!=i+1):
            if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
                    & (passed_state!=INTER) & ((pass_state_reqd==None) | (pass_state_reqd==passed_state) ):
#                 print(f'Matching client at {i+1} for seed:{cur_seed} nonce:{cur_nonce}')
                match=True
#                 rand = random.randint(1,2)
#                 print(f'random number:{rand}')
#                 if rand==1:
#                     match=True
            elif (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
#             print('Hash Chain starts')
            server_hash_list = generate_hash_chain(cur_hash,10)
            bincount_prev = compare_prev(server_hash_list,client_seed,cur_nonce,value_to_match=roll_hash)
#             print('Nonce Hash Chain starts')
            nonce_hash_list = generate_hash_chain(str(cur_nonce),10)
            bincount_prev_nonce = compare_prev(cur_hash,client_seed,nonce_hash_list,value_to_match=roll_hash)
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            bincount_row =[]
            for bincount_df in bin_counts_list:
                bincount_row.extend(list(bincount_df.values))
#             print(bincount_row)
            bincount_row_nonce =[]
            for bincount_df in bin_counts_list_nonce:
                bincount_row_nonce.extend(list(bincount_df.values))
#             print(bincount_row_nonce)
            result_success = ((passed_state==ABOVE) & (roll_seed_actual>5250))  \
                            | ((passed_state==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,total_passes,passed_state,
                                           total_passes_1,total_passes_2,
                                           roll_seed_actual,roll_hash,roll_blank_server] + \
                                            list(bincount_prev.values) + list(bincount_prev_nonce.values) +\
                                            bincount_row + bincount_row_nonce
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,total_passes,passed_state,roll_seed_actual,roll_hash}')
          
    return result_df
            
    

In [327]:
def predict_digit_pattern(server_hash,roll_hash,nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=None,mismatch_digit_indices=None):

#     rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
#     roll_first = rolls_list[0]
    factors = np.array([10000,1000,100,10])
    factors_filt = factors[match_digit_indices]
    for i,val in enumerate(factors_filt):
        match = match & (int((roll_hash % val) / (val / 10))==match_digit_arr[i])
        if not match: 
            break

    if mismatch_digit_arr is not None:
        factors_filt = factors[mismatch_digit_indices]
        for i,val in enumerate(factors_filt):
            match = match & (int((roll_hash % val) / (val / 10))!=mismatch_digit_arr[i])
            if not match: 
                break            

    return match
 

def predict_digit_output(server_hash,nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        hash_list_server,match_digit_indices=None):

    rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
    #eliminate zero from all roll_hash
#     for roll in rolls_list:
#         str_roll = f'{roll:04d}'
#         cur_count = str_roll.count('0')
#         if cur_count!=0:
#             match=False
            
    for roll,match_digit,match_count,match_indices \
            in zip(rolls_list,match_digit_arr,match_digit_count_arr,match_digit_indices):
        str_roll = f'{roll:04d}'
        
        if match_digit_indices is None:
            str_roll_indexed = str_roll
        else:
            str_roll_indexed = ''
            for index in match_indices:
                str_roll_indexed = str_roll_indexed + str_roll[int(index)]
                
        cur_count = str_roll_indexed.count(match_digit)
        if cur_count!=match_count:
            match=False
            
    return match, rolls_list

def validate_digits(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    feature_chain_length=20,
                    match_digit_indices=None,
                    last_index=None,result_df=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
        
            match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    if not match:
                        print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,pass_state_reqd,
                                           roll_seed_actual,rolls_list]
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df
  

In [328]:

def validate_test_topk_2(test,client_seed_data,hash_count_to_check,pass_state_reqd=BELOW,
                       last_index=None,result_df=None):
    i = len(client_seed_data)-1 #if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index','passed_state',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] )
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        cur_hash_list = generate_hash_chain(cur_hash,hash_count_to_check+1)
#         cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i] 
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hashes=[]
            for hash_chain_val in cur_hash_list:
                roll_hashes.append(calculate_roll(hash_chain_val,client_seed,cur_nonce))
            roll_hashes_arr = np.array(roll_hashes)
            
            roll_hashes_arr_2 = roll_hashes_arr[:len(roll_hashes_arr)-1]

            if pass_state_reqd==BELOW:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]<4750)
                mask = roll_hashes_arr_2<5250
#                 roll_hashes_arr_1stdigit = (roll_hashes_arr_2  / 1000).astype('int')
#                 flag_check_all_1000 = True
#                 for val1000 in range(6,10):
#                     flag_check_all_1000 = flag_check_all_1000 & len(roll_hashes_arr_1stdigit[roll_hashes_arr_1stdigit==val1000])>0
                    
            else:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]>5250)
                mask = roll_hashes_arr_2>4750
            
            match = (len(roll_hashes_arr_2[mask])==0) #& last_rev # & flag_check_all_1000
            roll_hash = roll_hashes_arr[0]
            i-=1
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,BELOW,
                                           roll_seed_actual,roll_hash,roll_blank_server] 
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success,cur_nonce,roll_seed_actual,roll_hashes_arr}')
          
    return result_df
            
    

In [329]:
# test_file_random ='/kaggle/input/hash-generate-random-seeds/test_data_random.csv'
# test_seeds_2=pd.read_csv(test_file_random)
# print(test_seeds_2.shape)
# test_seeds_2.head()

In [330]:
# test_file ='/kaggle/input/hashdata/server seed test data 4.csv'
# test_seeds_4=pd.read_csv(test_file)
# print(test_seeds_4.shape)
# test_seeds_4.head()

In [331]:
# test_file ='/kaggle/input/hashdata/server seed test data 5.csv'
# test_seeds_5=pd.read_csv(test_file)
# print(test_seeds_5.shape)
# test_seeds_5.head()

In [332]:
# #create special client seed pattern array

# def get_hexval(val):
#     hexdata='abcdef'
#     if val >=10:
#         return hexdata[val-10]
#     else:
#         return str(val)

# str_spl_client = 'woxpwoxpwoxpwoxpwoxp066103c1b2a6ebe01cf30afd49a6b931278793fc457dee84510f03e11779d5be'
# spl_client_list=[]
# for index in range(20,20+64):
#     for val in range(16):
#         spl_client_list.append(str_spl_client[:index]+get_hexval(val)+str_spl_client[index+1:])
# spl_client_array = np.array(spl_client_list)
# print(len(spl_client_array))
# spl_client_array[1020:]

##### Machine Learning

Generate Data

In [333]:
def generate_data(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    mismatch_digit_arr=None,mismatch_digit_indices=None,
                    feature_chain_length=20,
                    is_data_hash=False,
                    match_digit_indices=None,
                    match_count_exp=1,
                    test_limit = None,
                    last_index=None,result_df=None,
                    print_client_scan=False,
                    hash_list_nonce=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list','roll_hash_list_nonce',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        if (server_count % 5)==0:
            print(f'server_count:{server_count}')
        
        if (test_limit is not None) and (server_count >= test_limit):
            break
        
        #for large volume of data, it is recommended to set i to entire lengthof client seeds
        #for every server seed
#         i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
       
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length+1]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
          
        fullclientscan=False
        match = False
        prev_match=""
        match_count=0
        
        client_start = i
        while match_count < match_count_exp and not fullclientscan:
            
            if print_client_scan  & (i % 1000==0) :
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]
            client_index = i

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            match = predict_digit_pattern(cur_hash,roll_hash,cur_nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,
                        mismatch_digit_indices=mismatch_digit_indices)
        
            # match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
            #              match_digit_arr,match_digit_count_arr,
            #             cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if i==client_start and not match:
                print()
                print(i,client_start)
                print(f'No further match for seed:{cur_seed} nonce:{cur_nonce} match count: {match_count}')
                print()
                fullclientscan=True
            if (i<0):
                i = len(client_seed_data)-1
       
            if match:
                
#                 print(f'Matched for seed:{cur_seed} i:{i} match count: {match_count}')
                rolls_list  = compute_multirolls(cur_hash_list,client_seed,cur_nonce)
                if hash_list_nonce is None:
                    hash_list_nonce = generate_hash_chain(str(cur_nonce),feature_chain_length)

                rolls_list_nonce  = compute_multirolls_nonce(cur_hash,client_seed,hash_list_nonce)

                match_count += 1
                
                result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                                | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
                if result_success:
                    last_success_index= i+1
                result = [result_success,cur_seed, cur_hash, cur_nonce,
                                               client_seed,client_index,pass_state_reqd,
                                               roll_seed_actual,rolls_list,rolls_list_nonce]
                
                result_df.loc[len(result_df.index)] = result
#                 print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df


In [334]:
hashval = '88a0b251d848effe80f14afebff8576e6a7979cf1cc4ba5407b2c16d91387a79'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
nonce = 2670 #2390

feature_chain_length=60
train_hash_size = 2158#121
train_client_size = 2000 #20

In [335]:
analysis_df=None

In [336]:
cur_hash_list = generate_hash_chain(hashval,train_hash_size)

train_data = pd.DataFrame(columns=['seed','hash','nonce'])
start=24
train_data['seed']=np.array(cur_hash_list[start:-1])
train_data['hash']=np.array(cur_hash_list)[start+1:]
train_data['nonce'] = nonce

print(len(train_data))
train_data.head()

2134


,seed,hash,nonce
0,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670
1,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2e59b2d9560924bcffd1be70d7d654a00ddca8e5de463ce2eaff0c65bc84ea2b,2670
2,2e59b2d9560924bcffd1be70d7d654a00ddca8e5de463ce2eaff0c65bc84ea2b,547d560aeb978944dfe4eb835f4dfbb0d61468349be3eed35136ce936b1df614,2670
3,547d560aeb978944dfe4eb835f4dfbb0d61468349be3eed35136ce936b1df614,ce966c42f453f58853a80adf5e1b33c913885b61cffdc520c42f6a4aee068e76,2670
4,ce966c42f453f58853a80adf5e1b33c913885b61cffdc520c42f6a4aee068e76,d72dbe95d9f3f1f661aecb640e302613d9d21d6fdb45729540c744d892f8561e,2670


In [337]:
PATTERN_99=False
HIGH_ANALYSIS=False

cur_digit = 9
cur_zero_digit = 0

In [338]:
READ_FROM_FILE=False

In [339]:
%%time
# match_digit_arr = np.array(['999',])
# match_digit_count_arr=np.array([1])
# match_digit_indices = np.array(['012'])

if PATTERN_99:
    match_digit_arr=np.array([9,9])
    mismatch_digit_arr=np.array([9,9])
else:
    match_digit_arr=np.array([0,0])
    mismatch_digit_arr=np.array([0,0])
    
match_digit_indices = np.array([1,2])
match_digit_count_arr=np.array([1])
mismatch_digit_indices = np.array([0,3])

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 34.3 µs


In [340]:
%%time

if not READ_FROM_FILE:

    cur_client_seed_array = client_seed_array_2[:1000001]

    results_df = None

    last_index =  len(cur_client_seed_array)-1 #1024
    start = 0

    hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)

    results_df = generate_data(train_data,
                                cur_client_seed_array,
                                 ABOVE,
                        match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                        match_count_exp=train_client_size,
                        feature_chain_length=feature_chain_length,
                        is_data_hash = True,
                        test_limit=38,
                        last_index=last_index,result_df=results_df,
                        hash_list_nonce=hash_list_nonce)
    row = results_df.iloc[len(results_df)-1]
    last_index = row['client_index']

server_count:0
server_count:5
server_count:10
server_count:15
server_count:20
server_count:25
server_count:30
server_count:35
CPU times: user 45min 39s, sys: 1.28 s, total: 45min 40s
Wall time: 45min 49s


In [341]:
%%time

if not READ_FROM_FILE:

    cur_client_seed_array = client_seed_array_2[:1000001]

    results_df_temp = None

    last_index =  len(cur_client_seed_array)-1 #1024
    start = 0

#     hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)

    results_df_temp = generate_data(train_data,
                                cur_client_seed_array,
                                 ABOVE,
                        match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                        match_count_exp=train_client_size,
                        feature_chain_length=feature_chain_length,
                        is_data_hash = True,
                        test_limit=3,
                        last_index=last_index,result_df=results_df_temp,
                        hash_list_nonce=hash_list_nonce)


server_count:0
CPU times: user 59.8 s, sys: 36 ms, total: 59.9 s
Wall time: 60 s


In [342]:
# last_index
# results_df['client_index'].min()

In [343]:
# results_df.head()

In [344]:
# print(len(results_df))
# results_df['client_seed'].nunique()

In [345]:
%%time
if PATTERN_99:
    file_pattern_str = 'x99x'
else:
    file_pattern_str = 'x00x'
filename = f'data/results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
if READ_FROM_FILE:
    results_df = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                 'roll_hash_list_nonce': pd.eval})
else:
    results_df.to_csv(filename,index=False)

CPU times: user 2.18 s, sys: 69.6 ms, total: 2.25 s
Wall time: 2.31 s


In [346]:
def save_cleaned_results_df(results_df,is_test):
    nonce_suffix = "_nonce"
    rolls_df = results_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    rolls_df_nonce = results_df['roll_hash_list_nonce'].apply(pd.Series)
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df.shape[1])]
    results_df_cleaned = pd.concat([results_df,rolls_df,rolls_df_nonce],axis=1)
    results_df_cleaned=results_df_cleaned.drop(['roll_hash_list','roll_hash_list_nonce'],axis=1)
    if is_test:
        filename = f'data/cleaned_results_df_test_{nonce}_Large_{file_pattern_str}_pattern.csv'
    else:
        filename = f'data/cleaned_results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
    results_df_cleaned.to_csv(filename,index=False)
    return results_df_cleaned

In [347]:
results_df.head()

,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_hash_list,roll_hash_list_nonce
0,True,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxp57c5658a8d70e9fd2a0d2c675b5a70acb97aa8fc1cdcd4219e11649ab17f1810,999954,Above,6900,"[2002, 6561, 5599, 6525, 8447, 640, 2819, 6724, 9762, 1046, 7090, 2219, 7288, 9450, 2325, 3522, 8825, 8751, 2311, 4915, 6770, 3577, 2840, 753, 272, 8275, 2169, 1491, 5528, 5533, 2861, 97, 9750, 2651, 6463, 7088, 4618, 9545, 7066, 2148, 224, 9320, 217, 6604, 977, 8425, 8266, 6860, 9493, 8435, 6785, 2503, 1022, 3880, 6216, 1706, 7101, 685, 695, 3233, 5807]","[2002, 7481, 3052, 6007, 8888, 7258, 4311, 1865, 3331, 6034, 2468, 3881, 658, 4257, 2158, 4689, 1357, 9249, 2048, 6151, 4663, 3651, 5712, 7016, 9153, 3728, 5596, 752, 1170, 6976, 8634, 1115, 1522, 9457, 6752, 9491, 436, 1769, 6748, 7729, 6056, 8991, 5506, 4093, 2176, 2768, 2341, 8841, 5028, 5643, 7819, 7824, 1718, 5098, 1568, 6015, 6046, 3042, 4483, 3303, 1941]"
1,False,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxp665ef00f9273d1dd4adbb3a03b65d7669a899c06e67b16fd22ba542c589c74c4,999868,Above,1814,"[4007, 5251, 9819, 3640, 6205, 6393, 8132, 7264, 3532, 1469, 3865, 9827, 9732, 4350, 1057, 1021, 6645, 9932, 3702, 2595, 915, 844, 6739, 7321, 9525, 5277, 1427, 5522, 3079, 3566, 8687, 356, 9599, 2773, 5083, 3130, 3591, 2895, 5922, 4135, 3588, 5559, 7113, 6008, 5297, 522, 2854, 711, 7137, 1984, 6084, 6478, 7243, 2390, 5378, 1029, 5782, 6753, 7498, 3112, 85]","[4007, 7096, 3456, 1786, 5136, 5828, 457, 4135, 3810, 2909, 3238, 8014, 1371, 519, 2529, 7986, 7778, 8677, 774, 9873, 6163, 608, 222, 4036, 5119, 6115, 3427, 3587, 8600, 4804, 2908, 2388, 9070, 5024, 2011, 505, 6348, 8041, 9054, 1785, 8523, 933, 8425, 3394, 8010, 5567, 7575, 6246, 6311, 8248, 1820, 450, 1472, 1611, 1971, 9964, 3899, 2542, 3042, 3956, 2151]"
2,False,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxp944deeefc86f5718c79620717a012e82245786ee1d41db287db2337db08cdd2a,999764,Above,1892,"[8008, 6705, 7876, 2947, 5063, 5240, 4551, 556, 6984, 2304, 1213, 5693, 5826, 7069, 4674, 556, 700, 9059, 3700, 9977, 1450, 7750, 6476, 8165, 3291, 5650, 8560, 5420, 8, 8429, 9529, 3185, 6932, 4203, 5003, 5, 9954, 7659, 1509, 1355, 1350, 389, 3407, 8949, 6356, 6535, 2288, 7100, 3110, 8997, 9090, 4844, 5950, 8556, 1971, 4800, 8802, 1905, 2874, 3763, 7214]","[8008, 3884, 1257, 9618, 3198, 4351, 7551, 491, 5623, 1184, 6297, 550, 2988, 1837, 9566, 5923, 3456, 3459, 3416, 8078, 2854, 2357, 2133, 7347, 7210, 7262, 6006, 1619, 8504, 146, 6598, 4400, 3659, 1150, 8652, 4528, 7608, 4921, 7229, 4605, 9061, 9723, 3698, 1410, 4193, 5018, 89, 5697, 3532, 5065, 6811, 8134, 9188, 5627, 6563, 1026, 1573, 2550, 6519, 9084, 7513]"
3,False,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxpb383b822c75a0c1dfde875d81e587f84204202ce532f48ddd24d39483ed34ec8,999621,Above,2767,"[1006, 929, 4516, 1385, 952, 7031, 9498, 569, 9408, 5285, 3565, 4728, 5460, 7526, 2310, 6376, 7937, 7492, 8316, 4920, 4007, 6488, 5243, 7698, 209, 8683, 755, 7192, 4759, 5120, 3328, 1984, 3119, 9637, 478, 9772, 2220, 8496, 681, 1110, 733, 7162, 8455, 4415, 43, 8295, 2196, 431, 2338, 5658, 8288, 9555, 8833, 8422, 951, 8957, 5039, 7589, 7392, 4407, 303]","[1006, 8730, 3456, 2241, 1781, 5452, 9482, 1158, 9440, 2032, 772, 5274, 156, 9197, 9144, 1672, 4620, 3538, 1059, 9958, 1565, 168, 2197, 5522, 7624, 9481, 4043, 5296, 1219, 5227, 734, 7527, 1813, 8620, 3880, 5536, 3424, 8909, 7057, 9928, 1198, 3925, 1894, 3424, 7830, 7681, 2221, 986, 977, 7483, 1248, 5234, 2111, 5865, 7568, 4854, 2493, 8125, 2264, 8608, 6581]"
4,False,1c64436697

In [348]:
%%time
results_df_cleaned=save_cleaned_results_df(results_df,False)
results_df_cleaned.head()

CPU times: user 31.2 s, sys: 493 ms, total: 31.7 s
Wall time: 31.5 s


,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce
0,True,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxp57c5658a8d70e9fd2a0d2c675b5a70acb97aa8fc1cdcd4219e11649ab17f1810,999954,Above,6900,2002,6561,5599,6525,8447,640,2819,6724,9762,1046,7090,2219,7288,9450,2325,3522,8825,8751,2311,4915,6770,3577,2840,753,272,8275,2169,1491,5528,5533,2861,97,9750,2651,6463,7088,4618,9545,7066,2148,224,9320,217,6604,977,8425,8266,6860,9493,8435,6785,2503,1022,3880,6216,1706,7101,685,695,3233,5807,2002,7481,3052,6007,8888,7258,4311,1865,3331,6034,2468,3881,658,4257,2158,4689,1357,9249,2048,6151,4663,3651,5712,7016,9153,3728,5596,752,1170,6976,8634,1115,1522,9457,6752,9491,436,1769,6748,7729,6056,8991,5506,4093,2176,2768,2341,8841,5028,5643,7819,7824,1718,5098,1568,6015,6046,3042,4483,3303,1941
1,False,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxp665ef00f9273d1dd4adbb3a03b65d7669a899c06e67b16fd22ba542c589c74c4,999868,Above,1814,4007,5251,9819,3640,6205,6393,8132,7264,3532,1469,3865,9827,9732,4350,1057,1021,6645,9932,3702,2595,915,844,6739,7321,9525,5277,1427,5522,3079,3566,8687,356,9599,2773,5083,3130,3591,2895,5922,4135,3588,5559,7113,6008,5297,522,2854,711,7137,1984,6084,6478,7243,2390,5378,1029,5782,6753,7498,3112,85,4007,7096,3456,1786,5136,5828,457,4135,3810,2909,3238,8014,1371,519,2529,7986,7778,8677,774,9873,6163,608,222,4036,5119,6115,3427,3587,8600,4804,2908,2388,9070,5024,2011,505,6348,8041,9054,1785,8523,933,8425,3394,8010,5567,7575,6246,6311,8248,1820,450,1472,1611,1971,9964,3899,2542,3042,3956,2151
2,False,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191,2670,woxpwoxpwoxpwoxpwoxp944deeefc86f5718c79620717a012e82245786ee1d41db287db2337db08cdd2a,999764,Above,1892,8008,6705,7876,2947,5063,5240,4551,556,6984,2304,1213,5693,5826,7069,4674,556,700,9059,3700,9977,1450,7750,6476,8165,3291,5650,8560,5420,8,8429,9529,3185,6932,4203,5003,5,9954,7659,1509,1355,1350,389,3407,8949,6356,6535,2288,7100,3110,8997,9090,4844,5950,8556,1971,4800,8802,1905,2874,3763,7214,8008,3884,1257,9618,3198,4351,7551,491,5623,1184,6297,550,2988,1837,9566,5923,3456,3459,3416,8078,2854,2357,2133,7347,7210,7262,6006,1619,8504,146,6598,4400,3659,1150,8652,4528,7608,4921,7229,4605,9061,9723,3698,1410,4193,5018,89,5697,3532,5065,6811,8134,9188,5627,6563,1026,1573,2550,6519,9084,7513
3,False,1c64436697bd2af6b2d50ab06ea06cd024a9d9ec2f59ddec8d600ffdd63bd447,7b4f7fc7

In [349]:
HIGH_TARGET= 9000 #8944
LOW_TARGET = 1000 #1056
ZERO_COL_CUTOFF = 4

In [350]:
def create_target_mask(data,bHighCheck=HIGH_ANALYSIS):
    if bHighCheck:
        mask = data['roll_actual'] > HIGH_TARGET
    else:
        mask = data['roll_actual'] < LOW_TARGET   
    return mask

In [351]:
def generate_features(initial_df,istrain):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
#     for col in list(rolls_df.columns)[1:]:
#         rolls_df[f'{col}_dig2'] = ((rolls_df[col] % 1000) / 100).astype('int')
    rolls_df['count_lt_1000'] = (rolls_df < 1000).sum(axis=1)
    rolls_df['count_gt_5000'] = (rolls_df > 5000).sum(axis=1)-1
    
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual']],rolls_df],axis=1)
        mask = create_target_mask(features_df)
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
#         features_df=features_df.drop(['roll_actual'],axis=1)

        features_df['roll_hash_list_nonce']=initial_df['roll_hash_list_nonce']
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = rolls_df
    return features_df


In [352]:
train_manual = generate_features(results_df[25*train_client_size:50*train_client_size],True)
# train_manual = generate_features(results_df,True)
train_manual.tail()

0    23367
1     2633
Name: target, dtype: int64


,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_5000,target,roll_hash_list_nonce
75995,2413,2001,3938,698,7562,5728,2245,7537,6804,1753,4286,6204,3315,2549,8301,9190,8814,8903,2729,7740,5167,2040,2524,2242,4665,2812,9074,5975,9155,2470,9523,2702,2727,775,9971,8581,805,474,931,8287,5427,7428,793,6624,2916,8495,7045,8400,2249,4279,977,8741,2519,8866,1362,445,7592,2619,1065,8864,5733,5556,8,29,0,"[2001, 2099, 6084, 9342, 5009, 5599, 2913, 7234, 1958, 8861, 7751, 1602, 22, 4772, 7241, 6155, 5765, 1620, 5065, 8400, 4110, 4496, 8577, 678, 3810, 5369, 2850, 7163, 2364, 2796, 8903, 6788, 8858, 9424, 2243, 5370, 4292, 8639, 5640, 5216, 8770, 7468, 2635, 3686, 8453, 595, 1887, 9940, 2127, 9763, 4005, 1564, 4085, 6467, 9467, 9446, 5737, 8269, 631, 4235, 7968]"
75996,4730,2009,9875,3526,4038,9805,4176,5538,3306,5256,6249,7011,7912,2300,3866,8772,6449,9803,9369,3662,5817,698,8607,8019,9599,7301,2439,3000,8174,53,1268,7133,4050,2342,9115,4798,2160,1348,4029,1022,2043,3524,1244,4255,1470,3428,7627,7469,4978,9439,1900,1708,9531,6166,9595,5832,5020,560,8462,2019,7583,3316,3,28,0,"[2009, 9658, 91, 9826, 1440, 4594, 7585, 2531, 4777, 7036, 3477, 4046, 6656, 383, 7385, 867, 5736, 9998, 9938, 7196, 3719, 258, 1515, 8170, 7899, 4305, 654, 8219, 9375, 6468, 4366, 94, 3973, 1605, 4818, 627, 3197, 8492, 9051, 9714, 7534, 8812, 248, 4394, 6242, 2715, 9292, 8939, 2867, 3585, 6793, 7253, 6710, 1414, 8300, 2949, 89, 9076, 3126, 9218, 9605]"
75997,8993,5008,2281,7149,4609,1453,3293,9901,779,5791,208,2373,677,5236,3163,3487,9602,6089,7763,1998,8506,4953,2734,6901,19,2418,2639,1226,3235,620,9145,8647,6891,712,7895,35,7026,1905,8739,9696,1220,8907,2267,749,3517,1170,268,5348,29,7446,9127,955,1916,6222,1416,2482,9066,3167,6344,2288,9511,7620,11,25,0,"[5008, 1559, 7780, 4565, 6878, 8348, 6621, 1946, 7953, 1124, 2570, 998, 6723, 8097, 2811, 6752, 4725, 5464, 5976, 7210, 1182, 7590, 6485, 2773, 4854, 9500, 7363, 6188, 8153, 2055, 9068, 6088, 3648, 6119, 6510, 3292, 5338, 2242, 8830, 5065, 6558, 1907, 646, 9080, 6044, 4410, 2643, 8319, 6870, 9206, 3596, 813, 9584, 8348, 1340, 7809, 1138, 9713, 7126, 6324, 3062]"
75998,3314,4008,8034,2425,439,3074,1401,8629,6630,4300,2649,432,1304,7488,8913,5381,1510,5622,4287,2199,4144,7396,3198,875,2589,1201,676,959,3580,1848,889,5165,7229,7203,8439,9503,6910,7577,8354,5569,9970,3385,7627,6393,4621,3642,9572,9607,9226,2423,4839,179,2000,6409,9641,9551,6931,6573,9997,6202,4907,41,8,29,0,"[4008, 8071, 1842, 4689, 9868, 813, 1500, 8156, 205, 5236, 728, 8377, 2954, 215, 551, 3849, 6066, 5882, 6708, 9996, 2007, 1786, 845, 2005, 5584, 7393, 3497, 6021, 1215, 5920, 38, 2349, 5223, 5406, 2612, 4761, 1120, 7252, 3526, 4471, 7340, 1132, 154, 2130, 5381, 2814, 1381, 7378, 6973, 4302, 7677, 728, 8983, 8663, 8996, 5676, 2560, 4476, 7309, 7247, 1872]"
75999,9873,9004,9521,8418,679,9361,8131,5440,232,6160,2597,759,6854,5139,6261,9702,5206,8865,3785,1777,107,6774,8137,4929,8933,8738,2939,9266,784,8155,260,7758,6696,2067,8786,8423,2283,7060,3529,9483,5300,9004,4833,6486,6901,1437,830,5126,3957,9154,1177,4193,8164,1236,6999,3868,3844,7041,3679,7434,6652,8166,7,36,0,"[9004, 9942, 8341, 4132, 5936, 2905, 9361, 7707, 8780, 258, 5250, 6088, 8980, 8200, 7978, 5806, 878, 4314, 8578, 9964, 7528, 6665, 1318, 5498, 9414, 1761, 9411, 9487, 4255, 8610, 7138, 5604, 4563, 497, 8379, 7777, 3724, 7382, 2227, 3678, 8653, 9086, 8826, 2397, 813, 8451, 9081, 8808, 9124, 3828, 3776, 2071, 5260, 8746, 120, 2810, 4883, 7664, 6409, 6214, 353]"


In [353]:
cols_roll = [col for col in train_manual.columns if col.startswith('roll_') and col not in ['roll_0','roll_actual','roll_hash_list_nonce']]
print(cols_roll)

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']


In [354]:
def digit_9_stats(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual = (data['roll_actual']>HIGH_TARGET)
        
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual =(data['roll_actual']<LOW_TARGET)
        
    mask_actual = create_target_mask(data,bHighCheck)

        
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [355]:
def add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,ratio_min,
                aft_cur,tr_total,tr_success,
               aft_all,aft_0,aft_1,
               analysis_df):
    if aft_cur >=ratio_min: 
        result=[nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,aft_cur,np.nan,
               aft_all,aft_0,aft_1,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
               tr_total,tr_success,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
               ]
        analysis_df.loc[len(analysis_df.index)]=result  
    return analysis_df

def save_train_stats(data,nonce,zero_col_cutoff,pattern,bHighCheck,ratio_min,
                    analysis_df=None):
    
    digits = [9,8,7,6,5,4,3,2,1]
    for digit in digits:
#         print(f'****** digit : {digit}')
        if analysis_df is None:
            columns = ['nonce','zero_col_cutoff','pattern_99','target_high','digit','cutoff','afterval','tr_ratio','te_ratio',
                      'aft_all','aft_1','aft_2',
                       'pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5','pr_tr_cutoff','pr_val_cutoff',
                      'tr_total','tr_success','te_total','te_success',
                      'te_pr_1_count','te_pr_2_count','te_pr_3_count',
                       'te_pr_4_count','te_pr_5_count',
                      'te_mean_count','te_med_count','te_tr_count','te_val_count']
            analysis_df = pd.DataFrame(columns=columns)

        after_vals = [8,10]
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        mask_actual=create_target_mask(data,bHighCheck)
        for cutoff in cutoff_range:

#             print(f'********** cutoff:{cutoff}')
            mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff) 
            total_all = len(data[mask])
            if total_all ==0:
                success_all=0
                aft_all=0
            else:
                success_all = len(data[mask & mask_actual])
                aft_all =round(100 * success_all/ total_all ,0)
#             print(f'before values - probability: {aft_all} total : {total_all}  success: {success_all}')
            aft = np.zeros(len(after_vals))
            success = np.zeros(len(after_vals))
            total = np.zeros(len(after_vals))

            for i,val in enumerate(after_vals):
                mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
                total[i] = len(data[mask2])
                if total[i] ==0:
                    success[i]=0
                    aft[i]=0
                else:
                    success[i] = len(data[mask2 & mask_actual])
                    aft[i] =round(100 * success[i]/ total[i] ,0)
#                 print(f'AFTER - factor:{val} - probability: {aft[i]} total : {total[i]}  success: {success[i]}')

            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,25,ratio_min,
                                    aft_all,total_all,success_all,
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[0],ratio_min,
                                    aft[0],total[0],success[0],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[1],ratio_min,
                                    aft[1],total[1],success[1],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
    return analysis_df


In [356]:
# train_manual.head()

In [357]:
# results_df.iloc[1072]

In [358]:
train_manual['roll_actual_dig_1'] = (train_manual['roll_actual'] / 1000).astype('int')
print(train_manual['roll_actual_dig_1'].value_counts(normalize=True))

train_manual['count_hi'] = (train_manual[cols_roll] > 5250).sum(axis=1)
train_manual['count_lo'] = (train_manual[cols_roll] < 4750).sum(axis=1)

train_manual['count_dig_1'] = ( (train_manual[cols_roll] / 1000).astype('int')==1).sum(axis=1)
train_manual['count_dig_2'] = ( (train_manual[cols_roll] / 1000).astype('int')==2).sum(axis=1)
train_manual['count_dig_3'] = ( (train_manual[cols_roll] / 1000).astype('int')==3).sum(axis=1)
train_manual['count_dig_4'] = ( (train_manual[cols_roll] / 1000).astype('int')==4).sum(axis=1)
train_manual['count_dig_5'] = ( (train_manual[cols_roll] / 1000).astype('int')==5).sum(axis=1)
train_manual['count_dig_6'] = ( (train_manual[cols_roll] / 1000).astype('int')==6).sum(axis=1)
train_manual['count_dig_9'] = ( (train_manual[cols_roll] / 1000).astype('int')==9).sum(axis=1)
train_manual['count_dig_8'] = ( (train_manual[cols_roll] / 1000).astype('int')==8).sum(axis=1)
train_manual['count_dig_7'] = ( (train_manual[cols_roll] / 1000).astype('int')==7).sum(axis=1)
train_manual['count_dig_0'] = ( (train_manual[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
cols = ['roll_actual','roll_actual_dig_1',
        'count_dig_9','count_dig_0',
        'count_lt_1000','count_gt_5000','roll_0'] + cols_roll

1     0.102769
7     0.101962
6     0.101538
9     0.101500
0     0.101269
2     0.100231
8     0.100038
3     0.097269
4     0.097231
5     0.096154
10    0.000038
Name: roll_actual_dig_1, dtype: float64


In [359]:
# for cur_digit in [9,8,7,5]:
#     print()
#     print(f'************************** cur digit {cur_digit} ************************************************')
#     print('************************** HIGH ************************************************')
#     digit_9_stats(train_manual,True)
#     print()
#     print('************************** LOW ************************************************')
#     digit_9_stats(train_manual,False)

In [360]:
# %%time
# temp_df=None
# for zero_cutoff in range(0,6):
#     temp_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=temp_df)
# temp_df

Prediction

In [361]:
# # hashval = '393cc340566014a83e78d302f079c98f5cd4e4bc4c34321231cf479559556038'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
# seed = 'dummy' # hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
# # nonce = 2548 #2390

In [362]:
# test_data = pd.DataFrame(columns=['seed','hash','nonce'])
# test_data.loc[0,'seed']=seed
# test_data.loc[0,'hash']=hashval
# test_data.loc[0,'nonce'] = nonce

# print(len(test_data))
# test_data.head()

In [363]:
# if PATTERN_99:
#     match_digit_arr=np.array([9,9])
#     mismatch_digit_arr=np.array([9,9])
# else:
#     match_digit_arr=np.array([0,0])
#     mismatch_digit_arr=np.array([0,0])
    
# match_digit_indices = np.array([1,2])
# match_digit_count_arr=np.array([1])
# mismatch_digit_indices = np.array([0,3])


In [364]:
# %%time
# # match_digit_arr = np.array(['999',])
# # match_digit_count_arr=np.array([1])
# # match_digit_indices = np.array(['012'])

# # match_digit_arr=np.array([9,9])
# # match_digit_indices = np.array([1,2])
# # match_digit_count_arr=np.array([1])

# if not READ_FROM_FILE_TEST:

#     cur_client_seed_array = client_seed_array_2[:1000001]

#     results_df_test = None

#     last_index_test =  len(cur_client_seed_array)-1 #1024
#     start = 0

#     results_df_test = generate_data(test_data,
#                                 cur_client_seed_array,
#                                  ABOVE,
#                         match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
#                         mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
#                         match_count_exp=10000, #train_client_size,
#                         feature_chain_length=20,
#                         is_data_hash = False,
#                         test_limit=1,
#                         last_index=last_index_test,result_df=results_df_test,
#                         print_client_scan=False)
#     row = results_df_test.iloc[len(results_df_test)-1]
#     last_index_test = row['client_index']

In [365]:
# print(results_df_test['client_index'].min())
# len(results_df_test)

In [366]:
def generate_test_features(initial_df):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]

    print(cols_roll)
    rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

    rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
    rolls_df['roll_actual'] = initial_df['roll_actual']
    rolls_df['client_seed'] = initial_df['client_seed']
    rolls_df['client_index'] = initial_df['client_index']

    return rolls_df


In [367]:
def gen_test_data(cur_nonce,cur_hash,cur_seed,file_suffix=""):
    test_data = pd.DataFrame(columns=['seed','hash','nonce'])
    test_data.loc[0,'seed']=cur_seed
    test_data.loc[0,'hash']=cur_hash
    test_data.loc[0,'nonce'] = cur_nonce

    print(len(test_data))
    print(test_data.head())
    
    if PATTERN_99:
        match_digit_arr=np.array([9,9])
        mismatch_digit_arr=np.array([9,9])
    else:
        match_digit_arr=np.array([0,0])
        mismatch_digit_arr=np.array([0,0])

    match_digit_indices = np.array([1,2])
    match_digit_count_arr=np.array([1])
    mismatch_digit_indices = np.array([0,3]) 
    
    if PATTERN_99:
        file_pattern_str = 'x99x'
    else:
        file_pattern_str = 'x00x'
        
    filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    
    if  READ_FROM_FILE_TEST:
        results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                        'roll_hash_list_nonce': pd.eval})
    else:

        cur_client_seed_array = client_seed_array_2[:1000001]
        results_df_test = None

        last_index_test =  len(cur_client_seed_array)-1 #1024
        start = 0

        results_df_test = generate_data(test_data,
                                    cur_client_seed_array,
                                     ABOVE,
                            match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                            match_count_exp=10000, #train_client_size,
                            feature_chain_length=feature_chain_length,
                            is_data_hash = False,
                            test_limit=1,
                            last_index=last_index_test,result_df=results_df_test,
                            print_client_scan=False)
        #Remove client seed duplicates from test data
        results_df_test= results_df_test.drop_duplicates(subset='client_seed')
        
        results_df_test.to_csv(filename,index=False)
        
    print('Test Data Generation Completed')    
    test_manual = generate_test_features(results_df_test)
    print('Test Features Completed')    
        
#         row = results_df_test.iloc[len(results_df_test)-1]
#         last_index_test = row['client_index']   

    return results_df_test,test_manual
 

In [368]:
%%time
READ_FROM_FILE_TEST=False
# seed='dummy'
seed = generate_hash_chain(hashval,26)[25]
print('seed:',seed)
results_df_test,test_manual=gen_test_data(nonce,hashval,seed,file_suffix="")
print()
print(results_df_test[['seed','hash','nonce']].head(1))
test_manual.head()

seed: 7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191
1
                                                               seed  \
0  7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191   

                                                               hash nonce  
0  88a0b251d848effe80f14afebff8576e6a7979cf1cc4ba5407b2c16d91387a79  2670  
server_count:0

999999 999999
No further match for seed:7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191 nonce:2670 match count: 8014

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_4

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,6009,5934,9635,727,2449,8100,2876,816,693,4399,8446,2319,5533,960,6025,4686,3407,114,1127,278,4930,1201,8059,5170,6164,5567,1326,651,7985,6763,3033,3328,9318,8986,170,9068,325,7984,1412,3634,1821,8062,1324,1646,8714,9569,4226,9729,2793,2869,8186,8612,6037,6032,6279,1994,5282,8959,600,9306,1186,27,31,9,5,4,4,5,6,6,9,2,10,5567,woxpwoxpwoxpwoxpwoxp1e37a86b64fea1ab5fe6fd3d11d60244f7c3207f648fb1d402b555d7263ecc64,999978
1,1008,773,7291,6946,2587,9412,5390,4842,7978,9688,5823,5421,6583,1148,5722,1582,2518,1221,4083,1950,3380,4536,2026,5261,1122,4928,5558,4635,1527,3634,7761,9628,2807,6793,757,6003,1965,2279,5641,2513,9469,4554,5310,3935,5871,6639,9141,2648,8165,8339,8063,6440,572,3180,8750,4471,5056,5997,9321,5745,3247,30,27,7,7,5,7,12,6,6,4,3,3,4928,woxpwoxpwoxpwoxpwoxp95dbe6975f9858cf6ebb46c17fca06ac09ba9a5a9e54010a463916d214d48184,999884
2,4004,98,6665,5714,8844,8539,3669,3519,2030,9440,3163,6418,7580,4869,3386,4728,8749,8288,2993,2453,4981,4411,7402,2974,3306,9251,964,7941,2405,9246,9891,5382,8268,1408,351,5778,9905,1879,1738,1107,2377,9889,4721,3777,8118,7879,6829,333,7364,1619,2262,9247,4227,7361,1611,9399,1897,8095,2607,9135,286,28,30,7,8,6,6,3,3,9,7,6,5,9251,woxpwoxpwoxpwoxpwoxp0a120b9f4b927cb3f44b59527f7ebeb6c1fea747dc0a12ee057e526ab54a41eb,999806
3,3007,5152,4579,7341,9580,817,1832,8395,8865,5540,9381,6908,4724,4801,926,4550,8351,5747,4538,6514,8535,8330,4991,7739,6428,4950,521,3296,9536,3876,9696,1742,318,2198,7521,713,2956,1395,9120,9884,113,1135,9457,9389,7778,6251,6410,3653,8122,2731,597,6992,9700,3214,4442,3215,3465,1508,5078,76,7905,28,27,5,3,6,8,4,6,9,6,5,8,4950,woxpwoxpwoxpwoxpwoxp2d0fb9de4c48a76dda01685dbc07c8afe7c9bf58d2163d09a82650045c8716e1,999797
4,8001,2506,666,7881,3863,6441,821,1984,1322,7668,8636,5578,2375,1192,3930,1796,2671,8240,6343,3213,356,6841,5828,1634,4843,5051,5836,2709,5223,4980,6279,4192,5806,438,7269,7738,8658,8866,388,1360,6417,8215,7333,5608,2901,8040,1331,7016,797,14,1338,5132,5824,7506,1583,3983,2234,9615,850,8285,8393,27,28,9,6,4,3,9,5,1,8,7,8,5051,woxpwoxpwoxpwoxpwoxp270812fe08ed514d0f01ea5dc929e1534b852af245c6a3f17fd6a174ef9e0304,999779


In [369]:
%%time
READ_FROM_FILE_TEST=False
#Generate hash test data
print(f'{hashval=}')
hash_l2=generate_hash(hashval.encode())
print(f'{hash_l2=}')
results_df_test_hash,test_manual_hash=gen_test_data(nonce,hash_l2,hashval,file_suffix="_hash")
print()
print(results_df_test_hash.head(1))
test_manual_hash.head()

hashval='88a0b251d848effe80f14afebff8576e6a7979cf1cc4ba5407b2c16d91387a79'
hash_l2='274c875fc78a9959200182af3da4b425def7a8c8c75d8c7455c5343d3614a1ac'
1
                                                               seed  \
0  88a0b251d848effe80f14afebff8576e6a7979cf1cc4ba5407b2c16d91387a79   

                                                               hash nonce  
0  274c875fc78a9959200182af3da4b425def7a8c8c75d8c7455c5343d3614a1ac  2670  
server_count:0

999999 999999
No further match for seed:88a0b251d848effe80f14afebff8576e6a7979cf1cc4ba5407b2c16d91387a79 nonce:2670 match count: 8103

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,8006,7020,7947,6080,4206,1386,1937,7591,9019,7186,9248,4055,4255,3807,9393,8137,4600,9546,3325,2761,6729,26,7004,1628,3797,4123,9049,3497,6844,582,9366,7915,8579,5116,4158,1250,4023,1005,7973,1651,9734,9521,396,2887,3938,52,8482,5777,7048,5723,5216,3783,1300,3799,272,3062,1015,4314,6627,8139,3994,26,32,8,2,9,8,4,4,8,4,8,5,6543,woxpwoxpwoxpwoxpwoxpd2576198611ff99b78d320bf3ea0564fb1d820153f61c469d7ce15b00239d027,999899
1,5008,9543,3554,4888,3594,6157,7930,339,8716,4409,6840,3008,3783,4633,536,7776,3687,821,3260,608,9434,8817,7205,3502,381,1172,133,6466,3229,8966,8359,845,3117,103,8143,2427,6710,3057,5125,1568,8372,5188,8921,735,8860,718,2312,9284,1373,1266,488,6578,2877,2341,3522,9084,9515,2279,3754,8692,5409,23,34,4,5,12,3,3,5,5,9,3,11,3463,woxpwoxpwoxpwoxpwoxpf7f432ff4dbee68bf43e002c5d506124cb587873a8ca82111c305c8440c8be75,999896
2,7005,59,8892,4549,3304,8855,8028,8729,4521,674,6278,7791,1894,4786,3804,7779,1472,3925,3983,5664,888,1546,1947,7783,8840,320,6703,1090,285,66,9058,4264,3078,1157,1662,9060,4183,3784,5781,59,5074,1872,4662,2031,1369,2391,4873,5440,6,8695,6630,4705,6562,6767,8548,2732,8877,4024,5609,1698,8844,23,34,10,3,6,9,5,5,2,9,3,8,764,woxpwoxpwoxpwoxpwoxpe60026ef2fe7c3f33a3ad159800be68d366c2f7c264bbfa61464b3b448b84fbc,999874
3,9007,1396,2440,5640,7473,943,5500,5360,4189,5576,8278,8587,4597,4278,9183,5119,8651,8011,2185,845,3291,7462,6948,47,4865,269,5329,5020,4318,2032,3712,4486,916,8322,4948,2685,1693,8153,8563,1753,7299,2280,8748,7399,2633,6536,5448,5160,7377,5626,9158,9496,2146,1776,6171,6609,7560,4673,1003,9174,3208,29,26,5,7,3,8,10,4,4,8,6,5,7095,woxpwoxpwoxpwoxpwoxp52903b78a1b1ac8f7a98bf7e4026d3bae9307db14563dd6144fd8d50d7ac5dd4,999758
4,2003,5379,9438,2016,5520,7393,9446,2431,472,9476,2421,480,5652,7590,4496,9957,1665,3057,3110,6572,9017,7586,8666,8655,4204,137,3738,7776,6113,5298,9662,5008,1477,962,4641,1506,1966,7009,1395,6115,9181,7966,1512,7559,8894,3676,1354,6423,8284,6281,7435,607,2423,7956,2048,7521,5690,6137,8656,936,2411,33,26,7,6,4,3,6,6,7,5,10,6,9145,woxpwoxpwoxpwoxpwoxp604568f37ed864110564b83ada1382febea6fe97f7e5352002eacba20e14a30c,999632


In [370]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[25]
# cur_hash = cur_hash_list[26]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash25,test_manual_hash25=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash25")
# print()
# print(results_df_test_hash25.head(1))
# test_manual_hash25.head()

In [371]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[24]
# cur_hash = cur_hash_list[25]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash24,test_manual_hash24=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash24")
# print()
# print(results_df_test_hash24.head(1))
# test_manual_hash24.head()

In [372]:
%%time
results_df_test_cleaned=save_cleaned_results_df(results_df_test,True)
results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash25,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash24,True)

CPU times: user 7.27 s, sys: 59 ms, total: 7.33 s
Wall time: 7.32 s


In [373]:
# %%time
# if PATTERN_99:
#     file_pattern_str = 'x99x'
# else:
#     file_pattern_str = 'x00x'
# filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern.csv'

# if READ_FROM_FILE_TEST:
#     results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
#                                                         'roll_hash_list_nonce': pd.eval})
# else:
#     results_df_test.to_csv(filename,index=False)

In [374]:
# s=70
# e=80
# l=90
# client_seed_min_1=results_df[s*train_client_size:e*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[s*train_client_size:e*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)
# results_1 = results_df[s*train_client_size:e*train_client_size]
# print(results_1['client_index'].describe(percentiles=[0.09,0.1,0.15,0.2,0.25]))

# client_seed_min=results_df[e*train_client_size:l*train_client_size]['client_index'].min()
# client_seed_max=results_df[e*train_client_size:l*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# results_2 = results_df[e*train_client_size:l*train_client_size]
# print(results_2['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [375]:
# client_seed_min_1=results_df[25*train_client_size:39*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[25*train_client_size:39*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)

# client_seed_min=results_df[39*train_client_size:50*train_client_size]['client_index'].min()
# client_seed_max=results_df[39*train_client_size:50*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# print(results_df[39*train_client_size:50*train_client_size]['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# results_2 = results_df[39*train_client_size:50*train_client_size]
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [376]:
# mask = (results_df_test['client_index']<=client_seed_max_1) | (results_df_test['client_index']>=client_seed_min_2)
# results_df_test = results_df_test[mask]
# print(len(results_df_test))
# print(results_df_test['client_index'].describe(percentiles=[0.25,0.5,0.52,0.55,0.57,0.6,0.7,0.8,0.9]))

In [377]:
# test_manual = generate_test_features(results_df_test)

In [378]:
# # rand=random.randint(0,len(rolls_df)-1)
# rand=np.random.choice(range(0,len(rolls_df)))
# print('random location:',rand)
# rolls_df.iloc[rand]['client_seed']

In [379]:
# res = []
# for i in range(100):
#     res.append(np.random.choice(range(0,len(rolls_df))) )
    
# pd.Series(np.array(res)).value_counts()

In [380]:

def check_special_pattern_match(data,bHighCheck,cutoff,after_val):
     
#     rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
#     cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'

#     rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

#     rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
#     rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
#     rolls_df['roll_actual'] = initial_df['roll_actual']
#     rolls_df['client_seed'] = initial_df['client_seed']
#     rolls_df['client_index'] = initial_df['client_index']

    
    mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
    return data[mask]

def compareactual(rolls_df,bHighCheck):
    
    mask_actual=create_target_mask(rolls_df,bHighCheck)
    rolls_filt = rolls_df[mask_actual]
    return len(rolls_filt)


def save_test_stats(analysis_df, data,pattern,bHighCheck,is_prod=False):
    
    mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
    analysis_df_filt = analysis_df[mask]
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff
        
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
        data_filt = data[mask]
        total = len(data_filt)
        analysis_df.at[row.Index,'te_total']  = total  
        
        if not is_prod:
            success  = compareactual(data_filt,bHighCheck)
            if total ==0:
                prob=0
            else:
                prob = round(100 * success/total,0)

            analysis_df.at[row.Index,'te_ratio']  = prob  
            analysis_df.at[row.Index,'te_success']  = success  
        
    return analysis_df

        

In [381]:

# bHighCheck = False
# for cur_digit in [9,8,7,5]:
#     for bHighCheck in [True,False]:
#         if bHighCheck:
#             high_low_text = 'HIGH'
#         else:
#             high_low_text = 'LOW'
#         print()
#         print(f'****** cur_digit {cur_digit} ******  {high_low_text} ')

#         for cutoff in range(0,7):
#             rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,25)
#             len_match  = compareactual(rolls_df,bHighCheck)
#             if len(rolls_df) ==0:
#                 success_count=0
#             else:
#                 success_count = round(100 * len_match/len(rolls_df),0)
#             print()
#             print(f'cutoff:{cutoff} General --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')
#             for afterval in [8,10]:
#                 rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,afterval)
#                 len_match  = compareactual(rolls_df,bHighCheck)
#                 if len(rolls_df)==0:
#                     success_count=0
#                 else:
#                     success_count = round(100 * len_match/len(rolls_df),0)
#                 print(f'cutoff:{cutoff} afterval:{afterval} --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')



In [382]:
# analysis_df_test = save_test_stats(temp_df, test_manual)
# analysis_df_test

In [383]:
# mask = (analysis_df_test['digit'].isin([1,2,3,4,6])) & (analysis_df_test['tr_ratio']>=13)
# mask2 = mask & (analysis_df_test['te_ratio']>=13)
# print(len(analysis_df_test[mask]))
# print(len(analysis_df_test[mask2]))
# analysis_df_test[mask2]

##### Feature Generation

In [384]:
def generate_roll_features_k(rolls_df,roll_start,roll_end,suffix="",stat_suffix=""):
    first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    rolls_df[f'count_gt_9000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    rolls_df[f'count_lt_1000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] < 1000).sum(axis=1)
    
    return rolls_df
    
def generate_roll_features(rolls_df,
                           roll_start=1, roll_end=20,k=4,
                          suffix="",stat_suffix=""):
    cols_roll = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    print(cols_roll)
    
    if (roll_start==1) & (roll_end==20):
        stat_suffix = suffix
    else:
        stat_suffix = f'_{roll_start}_{roll_end}{suffix}'
    
    rolls_df[f'count_lt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
    rolls_df[f'count_gt_9000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df[f'count_gt_8000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df[f'count_gt_7000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df[f'count_gt_6000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df[f'count_gt_5000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df[f'count_gt_4000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df[f'count_gt_3000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df[f'count_gt_2000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df[f'count_gt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
    rolls_df[f'count_hi{stat_suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df[f'count_lo{stat_suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
    rolls_df[f'count_lo_hi{stat_suffix}'] = rolls_df[f'count_hi{stat_suffix}'] \
                                            + rolls_df[f'count_lo{stat_suffix}']   


    rolls_df=generate_roll_features_k(rolls_df,roll_start,roll_start+k,
                                      suffix=suffix,stat_suffix=stat_suffix)
    
    roll_list_strs = np.full(len(rolls_df),"")
    for col in cols_roll:
        roll_list_strs = rolls_df[col].astype('str') + roll_list_strs
    
    rolls_df[f'total_dig_9{stat_suffix}']= roll_list_strs.str.count('9')
    rolls_df[f'total_dig_0{stat_suffix}']= roll_list_strs.str.count('0')
    
    rolls_df[f'roll_mean{stat_suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
    rolls_df[f'roll_std{stat_suffix}'] = (rolls_df[cols_roll]).std(axis=1)
     
    print(f'Roll Features from {roll_start} to {roll_end} {suffix} completed' )
    
    return rolls_df

def generate_features_full(initial_df,istrain,feature_chain_length):
    
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]    
    rolls_df_nonce = initial_df['roll_hash_list_nonce'].apply(pd.Series)
    nonce_suffix = '_nonce'
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df_nonce.shape[1])] 
    
    #generate top 20 hash features
    rolls_df=generate_roll_features(rolls_df,roll_start=1,
                                    roll_end=20, k=4)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix,roll_start=1,
                                    roll_end=20 , k=4)
    
    #generate 25's hash features
    roll_start =25
    roll_end = 50
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)  

    #generate 50's hash features
    roll_start =50
    roll_end = 60
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)      
    #generate for full hash features
    roll_start =1
    roll_end = feature_chain_length
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)        
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                 rolls_df,
                                 rolls_df_nonce],axis=1)
        mask = create_target_mask(features_df)
            
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                             rolls_df,rolls_df_nonce],axis=1)
    return features_df


In [385]:
# def generate_roll_features(initial_df,colname,feature_chain_length,
#                            k=4,
#                           suffix=""):
#     rolls_df = initial_df[colname].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}{suffix}' for i in range(rolls_df.shape[1])]
    
#     # roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([str(i) for i in x]))
    
#     cols_roll = [f'roll_{i}{suffix}' for i in range(1,feature_chain_length+1)]
#     print(cols_roll)
    
# #     rolls_df[f'count_lt_1000{suffix}'] = (rolls_df < 1000).sum(axis=1)
# #     rolls_df[f'count_gt_9000{suffix}'] = (rolls_df >= 9000).sum(axis=1)
# #     rolls_df[f'count_gt_5000{suffix}'] = (rolls_df >= 5000).sum(axis=1)
# #     rolls_df[f'count_gt_7000{suffix}'] = (rolls_df >= 7000).sum(axis=1)
# #     rolls_df[f'count_gt_8000{suffix}'] = (rolls_df >= 8000).sum(axis=1)
    
#     rolls_df[f'count_lt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
#     rolls_df[f'count_gt_9000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df[f'count_gt_8000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df[f'count_gt_7000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df[f'count_gt_6000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df[f'count_gt_5000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df[f'count_gt_4000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df[f'count_gt_3000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df[f'count_gt_2000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df[f'count_gt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
#     rolls_df[f'count_hi{suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df[f'count_lo{suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
#     rolls_df[f'count_lo_hi{suffix}'] = rolls_df[f'count_hi{suffix}'] \
#                                             + rolls_df[f'count_lo{suffix}']   

#     first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(1,k+1)]
#     rolls_df[f'count_gt_9000_k{suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    
#     roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([f'{i:04d}' for i in x]))
    
#     rolls_df[f'total_dig_9{suffix}']= roll_list_strs.str.count('9')
#     rolls_df[f'total_dig_0{suffix}']= roll_list_strs.str.count('0')
    
#     rolls_df[f'roll_mean{suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
#     rolls_df[f'roll_std{suffix}'] = (rolls_df[cols_roll]).std(axis=1)
   

# #     if (suffix==""):
# #         rolls_df[f'roll_0_1st_digit'] = (rolls_df['roll_0'] / 1000).astype('int')
# #         rolls_df[f'roll_0_last_digit'] = (rolls_df['roll_0'] % 10)
    
    
#     return rolls_df

# def generate_features_full(initial_df,istrain,feature_chain_length):
    
#     rolls_df=generate_roll_features(initial_df,'roll_hash_list',feature_chain_length,k=4)
#     rolls_df_nonce=generate_roll_features(initial_df,'roll_hash_list_nonce',
#                                     feature_chain_length,k=4,suffix='_nonce' )
    
#     if istrain:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                  rolls_df,
#                                  rolls_df_nonce],axis=1)
#         mask = create_target_mask(features_df)
            
#         features_df['target'] =0
#         features_df.loc[mask,'target'] = 1
#         features_df['roll_actual']=features_df['roll_actual'].astype('int')
#         print(features_df['target'].value_counts())
#     else:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                              rolls_df,rolls_df_nonce],axis=1)
#     return features_df


In [386]:
# HIGH_ANALYSIS=True

In [387]:
%%time
# train = generate_features_full(results_df[train_client_size:],True,feature_chain_length)
train = generate_features_full(results_df,True,feature_chain_length)
print(len(train))
train.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [388]:
%%time
test = generate_features_full(results_df_test,False,feature_chain_length)
print(len(test))
test.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [389]:
%%time
test_hash = generate_features_full(results_df_test_hash,False,feature_chain_length)
print(len(test))
test_hash.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

#### Machine Learning

In [390]:
USE_STAT_FEATS = True

In [391]:

if USE_STAT_FEATS:
    exclude_stat_cols=[]
else:
#     exclude_stat_cols = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_col_starts = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_cols=[]
    for stat_col in exclude_stat_col_starts:
        exclude_stat_cols += [col for col in train.columns if col.startswith(stat_col)]
    print(exclude_stat_cols)
exclude_cols = []#[f'roll_{i}' for i in range(11,feature_chain_length)]
exclude_cols_nonce =[]# [f'roll_{i}_nonce' for i in range(11,feature_chain_length+1)]
exclude_count_cols = [f'count_gt_{i}000' for i in [1,2,3,4,6]]
exclude_count_cols_nonce = [f'count_gt_{i}000_nonce' for i in [1,2,3,4,6]]
othercols = ['roll_actual','client_seed','seed','hash'] + exclude_stat_cols

all_exclude_cols = othercols+exclude_cols+exclude_cols_nonce+\
                    exclude_count_cols+exclude_count_cols_nonce
                  
features = [col for col in train.columns if col not in all_exclude_cols]
print(len(features))
print(features)

265
['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60', 'count_lt_1000', 'count_gt_9000', 'count_gt_8000', 'count_gt_7000', 'count_gt_5000', 'count_hi', 'count_lo', 'count_lo_hi', 'count_gt_9000_k', 'count_lt_1000_k', 'total_dig_9', 'total_dig_0', 'roll_mean', 'roll_std', 'count_lt_1000_25_50', 'count_gt_9000_25_50', 'count_gt_8000_25_50', 'count_gt_7000_25_50', 'count_gt_6000_25_50', 'c

In [392]:
X = train[features].drop('target',axis=1)
y = train['target']

In [393]:
X.columns

Index(['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6',
       'roll_7', 'roll_8', 'roll_9',
       ...
       'count_gt_1000_1_60_nonce', 'count_hi_1_60_nonce',
       'count_lo_1_60_nonce', 'count_lo_hi_1_60_nonce',
       'count_gt_9000_k_1_60_nonce', 'count_lt_1000_k_1_60_nonce',
       'total_dig_9_1_60_nonce', 'total_dig_0_1_60_nonce',
       'roll_mean_1_60_nonce', 'roll_std_1_60_nonce'],
      dtype='object', length=264)

In [394]:
# results_df#[['nonce','roll_actual','roll_hash_list','client_seed','client_index']]

In [395]:
import xgboost as xgb
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score,precision_recall_curve

Train Validation Split

In [396]:
n_folds=10
tr_indices_folds = []
val_indices_folds=[]
tr_indices_seeds=[]
for fold in range(n_folds):
#     tr_indices = train[(fold+2) * train_client_size:].index
#     val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_offset = 1#25
    tr_length = 25#25
    tr_start = (fold+tr_offset) * train_client_size
    tr_end = (fold+tr_offset+tr_length) * train_client_size
    tr_indices = train[tr_start:tr_end].index #fold + 3
#     tr_indices = list(tr_indices) + list(train[70:90].index) #fold + 3
    
#     tr_indices_seed=[]
#     for i in range(1,tr_offset):
#         tr_start_seed = (fold+i) * train_client_size
#         tr_end_seed = (fold+i+1) * train_client_size
#         tr_indices_seed.append(train[tr_start_seed:tr_end_seed].index) #fold + 3 
        
    # tr_start_seed = (fold+1) * train_client_size
    # tr_end_seed = (fold+tr_offset) * train_client_size
    # tr_indices_seed = train[tr_start_seed:tr_end_seed].index #fold + 3
    
    tr_indices_seed = [train[0:(fold+1) * train_client_size].index]
#     val_indices = train[(fold+1) * train_client_size:(fold+2) * train_client_size].index
    val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_indices_seeds.append(tr_indices_seed)
    tr_indices_folds.append(tr_indices)
    val_indices_folds.append(val_indices)
    

In [397]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    
    f1_score = f1_score[:len(thresholds)]
    f1_score[np.isnan(f1_score)]=0
#     optimal_idx = np.nanargmax(precision[:len(precision)-1])
#     print(f'precision borders:{precision[0],precision[len(precision)-2]}')
#     print(f'recall borders:{recall[0],recall[len(recall)-2]}')
#     print(f'f1_score size:{len(f1_score)} recall size:{len(recall)}')
#     print(f'{precision=}')
#     print(f'{recall=}')
#     print(f'{f1_score=}')
#     print(f'{thresholds=}')
#     print(f'optimal_idx:{optimal_idx} precision size:{len(precision)} thresholds size:{len(thresholds)}')
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
#     return optimal_threshold, precision[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [398]:
ZERO_COL_CUTOFF=4

In [399]:
def train_status(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = 'count_gt_9000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
        cutoff_range = range(2+2,7+2)
        
    else:
        cutoff_col = 'total_dig_0'
        zero_col = 'total_dig_9'
        after_col = 'count_hi'
        cutoff_range = range(2,7)

    mask_actual=create_target_mask(data,bHighCheck)    
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        # print(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+train['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [400]:
def validate_pattern(initial_df,bHighCheck,zero_col_cutoff,digit,cutoff,after_val):
     
    # rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    # rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
    # cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    rolls_df= initial_df
    
    if bHighCheck:
        cutoff_col = f'count_gt_{digit}000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_lt_1000'
        zero_col = f'count_gt_{digit}000'
        after_col = 'count_hi'

    
    # rolls_df['client_seed'] = initial_df['client_seed']
    # rolls_df['client_index'] = initial_df['client_index']

    
    mask = (rolls_df[cutoff_col]==cutoff) \
                            & (rolls_df[after_col]<=after_val)  \
                            & (rolls_df[zero_col]==zero_col_cutoff)   \
                            #& (rolls_df['count_hi']+rolls_df['count_lo']<=18)
    
    return rolls_df[mask]
    

In [401]:
# val_res_df = validate_pattern(X_val,True,4,10)
# print(y_val[val_res_df.index])
# print(len(val_res_df))
# val_res_df

In [402]:
# probs= xgb_model.predict_proba(val_res_df)[:,1]
# print(probs)
# np.where(probs>0.3)

In [403]:
# # roll_actual_df = pd.DataFrame(columns=['roll_actual'])
# # roll_actual_df['roll_actual']=train['roll_actual'].iloc[tr_index]
# # print(len(X_tr))
# # print(len(roll_actual_df))

# oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# # fit and apply the transform
# roll_actual_df, y_temp = oversample.fit_resample(train[features+['roll_actual']].iloc[tr_index], 
#                                                  train['target'].iloc[tr_index])
# print(len(roll_actual_df))

# # tr_df = pd.concat([X_tr,roll_actual_df],axis=1)
# # print(len(tr_df))
# train_status(roll_actual_df,True)

In [404]:
%%time
READ_MODEL_FILE = False

if READ_MODEL_FILE:
    xgb_models = joblib.load(f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
else:

    params = { 'n_estimators':100,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              # 'scale_pos_weight': 5,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1}

    # xgb_model = xgb.XGBClassifier(**params)
    xgb_models = []
    scores  = []
    ratios =[]
    tr_cutoffs=[]
    val_cutoffs=[]
    tr_last_cutoffs=[]

    for fold,(tr_index, val_index, tr_index_seed) in enumerate(zip(tr_indices_folds,val_indices_folds,tr_indices_seeds)):
        X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
        X_val,y_val = X.iloc[val_index],y.iloc[val_index]

        oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
        # fit and apply the transform
        X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
        print(pd.Series(y_tr).value_counts())


        print(f'\n ******************* fold: {fold} ********')
        print(f'tr size: { len(X_tr)}  val size:  {len(X_val)}')
        print(f'{tr_index_seed}')
        print(f'{val_index}')
        print(f'{tr_index}')

        xgb_model = xgb.XGBClassifier(**params)
        xgb_model.fit(X_tr,y_tr)
        xgb_models.append(xgb_model)

        mean_opt_cutoff_tr = 0
        for tr_index_seed_cur in tr_index_seed:
            X_tr_seed,y_tr_seed = X.iloc[tr_index_seed_cur],y.iloc[tr_index_seed_cur]

            tr_probs = xgb_model.predict_proba(X_tr_seed)[:,1]
            opt_cutoff_tr, f1score = get_opt_cutoff_prec(y_tr_seed,tr_probs)
            # print('tr cutoff:',opt_cutoff_tr)
            tr_labels = convert_probtolabels(tr_probs,cutoff=opt_cutoff_tr) 
            mask1 = (tr_labels==1)
            mask2 = (y_tr_seed==1)
            total = len(tr_labels[mask1])
            success = len(tr_labels[mask1 & mask2])
            ratio = success/total if total!=0 else 0
            # print(f'tr ratio: {ratio} total: {total} success: {success}')
            mean_opt_cutoff_tr += opt_cutoff_tr / len(tr_index_seed)

        print(f'last cutoff :{opt_cutoff_tr}')
        print(f'mean tr cutoff :{mean_opt_cutoff_tr}')
        tr_last_cutoffs.append(opt_cutoff_tr)
        tr_cutoffs.append(mean_opt_cutoff_tr)
        val_probs = xgb_model.predict_proba(X_val)[:,1]
        print(val_probs[0:5])
        labels = convert_probtolabels(val_probs)
        score_init = accuracy_score(y_val,labels)

        opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_probs)
        print('valid cutoff:',opt_cutoff)
        val_cutoffs.append(opt_cutoff)
        # print(pd.Series(val_probs).describe(percentiles=[0.6,0.7,0.75,0.8]))
        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff_tr) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio: {ratio} total: {total} success: {success}')
        ratios.append(ratio)
        cur_f1 = f1_score(y_val, val_labels,average='macro')
        cur_acc = accuracy_score(y_val,val_labels)
        print(f'val accuracy score:{cur_acc} f1 score:{cur_f1:.4f} initial accuracy score:{score_init}')
        scores.append(cur_acc)

        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio @val_cutoff: {ratio} total: {total} success: {success}')

    #     train_status(X_tr,True)

    #     top_prob = np.sort(val_probs)[::-1][:1]
    #     top_label = y_val[val_probs==top_prob ]
    #     print(f'top_prob:{top_prob} top_label:{top_label}')

    print(f'mean score:{np.mean(scores)}')    
    print(f'mean ratio:{np.mean(ratios)}')   
    
    print(np.mean(np.array(val_cutoffs)))
    print(np.mean(np.array(tr_cutoffs)))
    print(np.mean(np.array(tr_last_cutoffs)))
    print(ratios)
    
    joblib.dump(xgb_models,f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
    print('Models Save completed')   

1    44982
0    44982
Name: target, dtype: int64

 ******************* fold: 0 ********
tr size: 89964  val size:  2000
[Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999],
           dtype='int64', length=2000)]
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999],
           dtype='int64', length=2000)
Int64Index([ 2000,  2001,  2002,  2003,  2004,  2005,  2006,  2007,  2008,
             2009,
            ...
            51990, 51991, 51992, 51993, 51994, 51995, 51996, 51997, 51998,
            51999],
           dtype='int64', length=50000)
last cutoff :0.1618099808692932
mean tr cutoff :0.1618099808692932
[0.45582095 0.45692536 0.48913074 0.43861988 0.486708  ]
valid cutoff: 0.16180998
ratio: 0.10810810810810811 total: 1998 success: 216
val accuracy score:0.1085 f1 sco

last cutoff :0.4247802495956421
mean tr cutoff :0.4247802495956421
[0.44686368 0.55574495 0.5018009  0.4968429  0.4706928 ]
valid cutoff: 0.39199784
ratio: 0.08791208791208792 total: 1547 success: 136
val accuracy score:0.276 f1 score:0.2615 initial accuracy score:0.7245
ratio @val_cutoff: 0.08872767857142858 total: 1792 success: 159
1    44943
0    44943
Name: target, dtype: int64

 ******************* fold: 8 ********
tr size: 89886  val size:  2000
[Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            17990, 17991, 17992, 17993, 17994, 17995, 17996, 17997, 17998,
            17999],
           dtype='int64', length=18000)]
Int64Index([16000, 16001, 16002, 16003, 16004, 16005, 16006, 16007, 16008,
            16009,
            ...
            17990, 17991, 17992, 17993, 17994, 17995, 17996, 17997, 17998,
            17999],
           dtype='int64', length=2000)
Int64Index([18000, 18001, 18002, 18003, 18004, 18005,

##### Prediction of Test 

In [405]:
# results_df[:train_client_size].head()

In [406]:
# results_df_test.head()

In [407]:
# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     for row in analysis_df.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for model_ind in model:
#                 probs+= model_ind.predict_proba(test_filt_df[features_test])[:,1] / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#         prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]         
#     return analysis_df


In [408]:
# def save_test_ml_probs_internal(analysis_df_filt,
# #                                 features_test,
# #                                 test,model,
#                                 prob_tr_cutoff,prob_val_cutoff,
#                                 pattern,bHighCheck,multi_models):
    
#     start = time.time()
#     #get data from shared memory
#     features_test = p.value[0]
#     test = p.value[1]
#     model = p.value[2]
# #     print(len(test))
# #     print(test.head())
#     end = time.time()
#     print(f'Read Shared memory time:{end-start}')
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df_filt.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df_filt.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df_filt.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df_filt.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df_filt.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df_filt.at[row.Index,'pr_val_cutoff']  = ratios[8]       

#     end = time.time()
#     print(f'Internal function time:{end-start}')
#     return analysis_df_filt  

# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
#     other_args=[#features_test,test,model,
#                 prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,multi_models]
#     analysis_df_filt=parallelize_dataframe(analysis_df_filt, save_test_ml_probs_internal, 
#                                            data_to_share=[features_test,test,model],
#                                            other_args=other_args, n_cores= multiprocessing.cpu_count())
#     if len(analysis_df_filt)==len(analysis_df):
#         analysis_df=analysis_df_filt
#     else:
#         analysis_df=pd.concat([analysis_df[~mask],analysis_filt],axis=0)
#     return analysis_df

In [409]:
# temp_df = pd.DataFrame()
# temp_df['one'] = np.array([3,42,20,8,19,54,72,15])
# temp_df['two'] = np.array([0,1,3,1,1,2,2,3])

# temp_df_filt = temp_df[temp_df['one']>=42]
# for row in temp_df_filt.itertuples():
#     print(row.Index)
# #     row.loc[row.Index,'two']=5
#     temp_df.loc[row.Index,'two']=5
# print(temp_df_filt.head())    
# temp_df

In [410]:
# def save_test_ml_probs_old(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]  
        
#     end = time.time()
#     print(f'Function time:{end-start}')   

#     return analysis_df

In [411]:
MEAN_CUTOFF=-1
MEDIAN_CUTOFF=-2
TR_CUTOFF=-3
VAL_CUTOFF=-4


def save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                      analysis_df,row_index,
                      bHighCheck):
    probs = test_filt_df['probs']
    prob_cutoffs = [np.mean(probs),
#                     np.percentile(probs,0.75),
                    np.median(probs),
                    0.1,0.2,0.3,0.4,0.5,
                    prob_tr_cutoff,prob_val_cutoff]
    ratios = np.zeros(len(prob_cutoffs))
    total = np.zeros(len(prob_cutoffs))
    for i,prob_cutoff in enumerate(prob_cutoffs):
        test_filt_prob=test_filt_df.loc[probs>prob_cutoff]
#         indices= np.where(probs>prob_cutoff)[0]
#         roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
        total[i] = len(test_filt_prob)
        mask=create_target_mask(test_filt_prob,bHighCheck)
        success = len(test_filt_prob[mask])
        if total[i]==0:
            ratios[i]=0
        else:
            ratios[i] = 100 * success / total[i]

        if (i >= 2) & (i<=6):
            analysis_df.at[row_index,f'pr_{i-1}']  = ratios[i] 
            analysis_df.at[row_index,f'te_pr_{i-1}_count']  = total[i] 

#             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')

    analysis_df.at[row_index,'pr_mean']  = ratios[0]  
    analysis_df.at[row_index,'pr_med']  = ratios[1] 
    analysis_df.at[row_index,'pr_tr_cutoff']  = ratios[7]  
    analysis_df.at[row_index,'pr_val_cutoff']  = ratios[8]  
    
    analysis_df.at[row_index,'te_mean_count']  = total[0]  
    analysis_df.at[row_index,'te_med_count']  = total[1] 
    analysis_df.at[row_index,'te_tr_count']  = total[7]  
    analysis_df.at[row_index,'te_val_count']  = total[8]
    
    return analysis_df


def map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff):
    
    if prob_cutoff==MEAN_CUTOFF:
        prob_cutoff=np.mean(probs)
    elif prob_cutoff==MEDIAN_CUTOFF:
        prob_cutoff=np.median(probs)
    elif prob_cutoff==TR_CUTOFF:
        prob_cutoff=prob_tr_cutoff
    elif prob_cutoff==VAL_CUTOFF:
        prob_cutoff=prob_val_cutoff
        
    return prob_cutoff

def generate_probs(multi_models,model,data,features):
    if multi_models:
        probs= np.zeros(len(data))
        for i,model_ind in enumerate(model):
            cur_prob= model_ind.predict_proba(data[features])[:,1] 
            probs+= cur_prob / len(model)
    else: 
        probs= model.predict_proba(test_filt_df[features_test])[:,1]
    data['probs']=probs
    return data

def save_test_ml_probs(analysis_df,test,model,
                       prob_tr_cutoff,prob_val_cutoff,
                       pattern,bHighCheck,
                       gen_probs=True,
                       is_prod=False,
                       prob_cutoff=-1,
                      multi_models=False):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
#         print(f'{filt_size=}')
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue
        if gen_probs:
            test_filt_df = generate_probs(multi_models,model,test_filt_df,features_test)
        probs = test_filt_df['probs'].values
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]

        if is_prod:
            prob_cutoff=map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff)
            mask = (test_filt_df['probs'] > prob_cutoff)
            test_filt_df=test_filt_df[mask]
        
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
        if not is_prod:
            analysis_df = save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                              analysis_df,row.Index,bHighCheck)
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all,analysis_df

In [412]:
# features_test = [col for col in features if col!='target']
# test_filt_df = validate_pattern(test[features_test],True,4,9,0,8)

# roll_filt = test['roll_actual'].loc[test_filt_df.index]
# total = len(roll_filt)
# if HIGH_ANALYSIS:
#     success = len(roll_filt[roll_filt>=9000])
# else:
#     success = len(roll_filt[roll_filt<1000])
# if total==0:
#     ratio=0
# else:
#     ratio = success / total
# print(f'ratio:{ratio} total:{total} success:{success}')
# # test_filt_df

In [413]:
# probs= xgb_model.predict_proba(test_filt_df)[:,1]
# # print(probs)
# print(np.mean(probs))
# print(np.median(probs))
# prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5]

# for prob_cutoff in prob_cutoffs:
#     indices= np.where(probs>prob_cutoff)[0]
# #     print(len(indices))
#     # print(indices)
#     # print(test_filt_df.iloc[indices].index)
#     roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#     total = len(roll_filt)
#     if HIGH_ANALYSIS:
#         mask = roll_filt>=9000
#         mask_arr =roll_filt.values>=9000
#     else:
#         mask=roll_filt<1000
#         mask_arr =roll_filt.values<1000
# #     print(roll_filt.tail())
#     success = len(roll_filt[mask])
# #     print(np.where(mask_arr))
#     if total==0:
#         ratio=0
#     else:
#         ratio = success / total
#     print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')


In [414]:
CONT_ANALYSIS=False
IS_PROD=False

In [476]:
if not IS_PROD:
    
    actual_seed="b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263"
#     actual_seed = cur_hash_list[25]
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(actual_seed,
                                           results_df_test['client_seed'],
                                           nonce)

    results_df_test['roll_actual']=roll_array
    test_manual = generate_test_features(results_df_test)
    test = generate_features_full(results_df_test,False,feature_chain_length)

b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 

In [477]:
l1=list(range(1,261))
# l2=list(range(50,80))
for hash_idx in [-1]+l1:
    
    if hash_idx==-1:
        actual_seed="b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263"
        suffix = 'orig'
    else:    
        actual_seed = cur_hash_list[hash_idx]
        suffix = hash_idx
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    test[f'roll_actual_{suffix}']=vectorized_calculate_roll(actual_seed,
                                           test['client_seed'],
                                           nonce)

b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263
274c875fc78a9959200182af3da4b425def7a8c8c75d8c7455c5343d3614a1ac
30d1640c8b519c56fda831b0184a790236bf9e008734b09aeac4fe021b92eaba
6b083ba1fff8ea8283ff26e78f9f2d7792a385edc54a8904bbcdf8fb3e09c1cf
2440ccb5014aa6a3fef0546922f36813e54042768193c765dd037dae5e0889be
6c4d2e541047fb82103b9a5afaaca2702e0540b91f9a0042344555dee9d24450
c7dff9c65e971abf39967b479e01a973c3311d17f6d81a40838f7a81d1bdbd11
a95249bb7213721df44be982aef547db3703bc78abab112c0e5f4ff77af9480f
f2948e1f276a8e0e9841a63ed5504d7ce73673a1d178b2b429900bcc4a7b48b7
ebd8d88150732bd55ed39b268eb166fab7f673e983f3ef8822ff5403fb2ce8bc
743dbe4a2125cbc9a0b2291ed93c0e4700963ddd00bb24f303147264711534eb
811027f051431b154c213a4904df8ce6245b8c99bb93c6b96714ccdc487148b9
0269be5714f622466ed12ed05bec549fe492773befb103019a341dcef38b4729
b89bb4eadfd198e60bfa47028733a2eb291e18a1c7503cbabd0678388da53ab8
d264a43846cdd7deeff24860a05cdb578ed6e0e15932bcf38ee49eb8d9359298
5ee87e19f94902202c1338a9c

9adec7bebc9ef27c09589420017452f908de47a52f5b4eb5ae054f5ec384abe3
40b6164e35b09d870202521f71d8520f3ead198486ac1524ead173d49c142a9e
b63f12585e1346b653d58b988869e0b4d08f7a391f048351b07cfa824dfc2e98
4742d3670921e860c01a653222ba47a75902ad80a529e7fd473cea767f0279a9
d4be093ee9f7b10e261e88c22f7ea193472bc000164361bddaa3425b2f3ad4d3
91ce6d3f993c9ec7d8025a5440159c4d8263027cb7cd1d55f2fd6c7dcdc66da1
02769260065d10ff3d74c9f7fb573561a3ddfe0bdb5e7ef25de913abd3a11e70
3e3d708fdd05904751429a4e7e9c83145650399de90584ca8ea7d2ec4e3b7c6a
10f9415ca6f25b6301bf9df0d6fcad2cd290ed3df2321150988d30ca9cda713c
eef1a83d54b52770b28f09a52cd9c380507f8f84e3d509b4c09f0199d6801800
3d848c49258af19f902e27359305cdd2491234d65a2e10c062d30c4882912ccd
fc6875f6238ec64f5da93815da361f598b370188a8720816d3538e0a7b6b6161
ea48ed6ae128e7d222e0a3e5f519f2604219651f8274fdf0f0a29cc513e0884c
8d5dbfebbaf55d05b970ef67ad7c85cc5ed63ab0558df12abdad9fd477e5716d
67d303272b1109de75b37a7ab164e0e579bd01a7d867970c1c54822e62dc5a99
6b7b72c23177f5deb76574608

f9045d88e3f4e0ef8aef495aad8837e36789eb3288bf540688b05955d94273dd
06dbc217ab516fb41b093fa89f78c049e84438d53852a0a4650718fa9aa3176e


In [417]:
# analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')

In [418]:
mean_imp = np.zeros(len(features)-1)
imp_df = pd.DataFrame()
imp_df['feature'] = [col for col in features if col!='target']
for i,model in enumerate(xgb_models):
    cur_imp = model.feature_importances_
    imp_df[f'importance_m{i}'] = cur_imp
#     print(cur_imp)
    mean_imp += cur_imp / len(xgb_models)

# print(mean_imp)
imp_df['importance'] = mean_imp
imp_df = imp_df.sort_values('importance',ascending=False).reset_index(drop=True)
imp_df

,feature,importance_m0,importance_m1,importance_m2,importance_m3,importance_m4,importance_m5,importance_m6,importance_m7,importance_m8,importance_m9,importance
0,count_hi_50_60,0.004845,0.005152,0.004486,0.004088,0.003321,0.005367,0.005467,0.004603,0.003655,0.004848,0.004583
1,roll_37,0.004561,0.004549,0.005121,0.004433,0.004823,0.004732,0.004515,0.004032,0.004319,0.004732,0.004582
2,count_lt_1000_k_1_60_nonce,0.006069,0.006260,0.000000,0.006013,0.005906,0.002740,0.007265,0.005220,0.002060,0.003943,0.004548
3,roll_45_nonce,0.004676,0.004572,0.003593,0.004786,0.004617,0.004184,0.004733,0.004504,0.004743,0.004588,0.004500
4,roll_mean_50_60,0.004832,0.005094,0.004759,0.004732,0.003740,0.004706,0.004393,0.004081,0.004044,0.004516,0.004490
5,roll_54_nonce,0.004449,0.004517,0.004303,0.004309,0.004590,0.004450,0.004337,0.004205,0.004692,0.004986,0.004484
6,roll_53_nonce,0.004144,0.003708,0.004311,0.004415,0.004344,0.004402,0.004774,0.005253,0.004712,0.004515,0.004458
7,roll_std_50_60_nonce,0.004340,0.004352,0.004436,0.004782,0.004406,0.004270,0.004291,0.004051,0.004859,0.004670,0.004446
8,roll_57_nonce,0.004392,0.004427,0.004774,0.004183,0.004570,0.004532,0.004510,0.004639,0.003981,0.004205,0.004421
9,roll_33_nonce,0.004268,0.004715,0.005111,0.004407,0.004179,0.004212,0.004133,0.004145,0.004952,0.004075,0.004420


In [419]:
# %%time
# test_hash25 = generate_features_full(results_df_test_hash25,False,feature_chain_length)
# test_hash25.head()

In [420]:
# %%time
# test_hash24 = generate_features_full(results_df_test_hash24,False,feature_chain_length)
# test_hash24.head()

In [421]:
test_hash1 = test_hash.copy()
# test_hash = test_hash24.copy()
# test_hash = test_hash1.copy()

In [422]:
test_filt=test.copy()
test_filt_hash=test_hash.copy()
features_test = [col for col in features if col not in ['target','probs']]

In [450]:
test_hash_probs = generate_probs(True,xgb_models,test_hash,features_test)
cutoffs = []
ratios =[]
totals =[]
print(test_hash_probs['probs'].describe())
cutoffs = np.linspace(test_hash_probs['probs'].min(),
                      test_hash_probs['probs'].quantile(0.95),700)
for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):
#     cutoff=0.4
#     diff = 0.01
#     print(cutoff,cutoffs[i+1])
    mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
    mask2=create_target_mask(test_hash_probs)
    
#     mask2 = test_hash_probs['roll_actual']>=9000
    total = len(test_hash_probs[mask])
    if total==0:
        ratio=0
        success=0
    else:
        success = len(test_hash_probs[mask & mask2])
        ratio = success / total
    ratios.append(ratio)
    totals.append(total)
#     print(ratio,success,total)

print(len(ratios),len(cutoffs))
df=pd.DataFrame()
df['cutoff']=cutoffs[:len(cutoffs)-1]
df['cutoff_2']=cutoffs[1:len(cutoffs)]
df['ratio'] =ratios
df['total'] =totals

df = df.sort_values('ratio',ascending=False).reset_index(drop=True)
print(df[:5].mean())
print(df[:10].mean())
df.head(10)

count    8103.000000
mean        0.457962
std         0.036390
min         0.269399
25%         0.438423
50%         0.463464
75%         0.482846
max         0.555845
Name: probs, dtype: float64
699 700
cutoff      0.337024
cutoff_2    0.337365
ratio       1.000000
total       1.000000
dtype: float64
cutoff      0.352889
cutoff_2    0.353230
ratio       0.896667
total       1.800000
dtype: float64


,cutoff,cutoff_2,ratio,total
0,0.269399,0.269740,1.000000,1
1,0.384723,0.385064,1.000000,1
2,0.338320,0.338661,1.000000,1
3,0.339344,0.339685,1.000000,1
4,0.353333,0.353674,1.000000,1
5,0.357086,0.357427,1.000000,2
6,0.334567,0.334908,1.000000,1
7,0.385405,0.385746,0.800000,5
8,0.392229,0.392570,0.666667,3
9,0.374487,0.374828,0.500000,2


In [451]:
df

,cutoff,cutoff_2,ratio,total
0,0.269399,0.269740,1.0,1
1,0.384723,0.385064,1.0,1
2,0.338320,0.338661,1.0,1
3,0.339344,0.339685,1.0,1
4,0.353333,0.353674,1.0,1
...,...,...,...,...
694,0.311707,0.312048,0.0,0
695,0.379605,0.379946,0.0,2
696,0.379263,0.379605,0.0,6
697,0.312048,0.312389,0.0,1


In [452]:
df.describe()

,cutoff,cutoff_2,ratio,total
count,699.000000,699.000000,699.000000,699.000000
mean,0.388476,0.388817,0.071163,11.011445
std,0.068897,0.068897,0.137261,12.650578
min,0.269399,0.269740,0.000000,0.000000
25%,0.328937,0.329278,0.000000,0.000000
50%,0.388476,0.388817,0.000000,5.000000
75%,0.448014,0.448355,0.101724,20.000000
max,0.507553,0.507894,1.000000,50.000000


In [454]:
print(len(df[df['total']==1]))
print(len(df[(df['total']==1) & (df['ratio']>0)]))
df[df['total']==1]

75
6


,cutoff,cutoff_2,ratio,total
0,0.269399,0.269740,1.0,1
1,0.384723,0.385064,1.0,1
2,0.338320,0.338661,1.0,1
3,0.339344,0.339685,1.0,1
4,0.353333,0.353674,1.0,1
6,0.334567,0.334908,1.0,1
382,0.298400,0.298742,0.0,1
390,0.278270,0.278611,0.0,1
432,0.316825,0.317166,0.0,1
433,0.353674,0.354015,0.0,1


In [427]:
df[df['ratio']<0.12].index[0]

147

In [428]:
test['roll_actual']=test[f'roll_actual_orig'].copy()

In [478]:
test_probs = generate_probs(True,xgb_models,test,features_test)
top_k=[]
all_k=[]
for k in range(len(df)):
    row = df.iloc[k]
#     print(row['ratio'],row['cutoff'],row['cutoff_2'])
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    mask2 = create_target_mask(test_probs)    
    test_filt = test_probs[mask]

    total = len(test_filt)
    if total==0:
        success=0
        ratio=0
    else:
        success = len(test_probs[mask & mask2])
        ratio = success / total
    if total==1:
        all_k.append(k) 
    if ratio>=0.12:
        if total==1:
            top_k.append((k,ratio,total))
#     print(ratio,success,total,row['cutoff'],k)
print(len(all_k))
print(all_k)
print(len(top_k))
print(top_k)
mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
test_filt_hash  = test_hash_probs[mask]

77
[3, 4, 6, 10, 11, 12, 13, 14, 15, 26, 65, 82, 91, 320, 327, 343, 344, 351, 365, 389, 411, 426, 428, 431, 432, 433, 435, 440, 442, 443, 444, 447, 448, 450, 455, 460, 462, 463, 468, 469, 479, 486, 488, 489, 492, 493, 496, 500, 502, 508, 510, 515, 517, 520, 521, 530, 536, 541, 542, 544, 547, 550, 553, 556, 560, 561, 566, 578, 644, 653, 654, 663, 671, 681, 690, 691, 695]
10
[(13, 1.0, 1), (91, 1.0, 1), (320, 1.0, 1), (435, 1.0, 1), (442, 1.0, 1), (463, 1.0, 1), (508, 1.0, 1), (541, 1.0, 1), (644, 1.0, 1), (690, 1.0, 1)]


In [479]:
all_k_arr = np.array(all_k)
diff = np.diff(all_k_arr)
print(diff)
ind = np.where(diff>=20)
print(ind[0]+1)
trans_k = np.sort(np.concatenate((all_k_arr[ind[0]],all_k_arr[ind[0]+1])))
print(len(trans_k))
trans_k

[  1   2   4   1   1   1   1   1  11  39  17   9 229   7  16   1   7  14
  24  22  15   2   3   1   1   2   5   2   1   1   3   1   2   5   5   2
   1   5   1  10   7   2   1   3   1   3   4   2   6   2   5   2   3   1
   9   6   5   1   2   3   3   3   3   4   1   5  12  66   9   1   9   8
  10   9   1   4]
[10 13 19 20 68]
10


array([ 26,  65,  91, 320, 365, 389, 389, 411, 578, 644])

In [457]:
def get_sim_cutoff_data(test,xgb_models,features_test,test_hash_cutoff_df,
                       is_imp_feats,imp_feats_count,gen_probs=True):
    if gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test)
    else:
        test_probs = test
    print('Generate probs completed')
    test_probs['roll_actual_init']=test_probs['roll_actual'].copy()

    if is_imp_feats:
        hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count] ]
#         hash_groups = [['roll_mean_nonce']]
    else:
    #     hash_groups = [['roll_21_nonce', 'roll_37_nonce', 'roll_mean_nonce','roll_36_nonce','roll_48',
    #                    'roll_53_nonce','roll_32_nonce','roll_44_nonce','roll_38_nonce','roll_17']]
        hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
#         hash_groups = [list(range(25,36)) ]
    hash_groups_flat = [item for sublist in hash_groups for item in sublist]
    print(hash_groups)
    print('list creation complete')
    

    top_k=[]
    sim_cutoff_df = pd.DataFrame(columns=['k','ratio','total','hash_no','hash_group','roll_mean'])
    df_size = len(test_hash_cutoff_df)
    for i,row in enumerate(test_hash_cutoff_df.itertuples()):
        k=row.Index
#         if (k%10)==0:
#             print(f'loop for k = {k}')
#         row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row.cutoff) & (test_probs['probs']<=row.cutoff_2)
        test_filt = test_probs[mask]
        total = len(test_filt)
        ratios=[]

        for idx in hash_groups_flat:
#             print(f'k:{k} idx:{idx}')
            if is_imp_feats:
                test_filt['roll_actual']=test_filt[idx].copy()    
            else:
                test_filt['roll_actual']=test_filt[f'roll_actual_{idx}'].copy()
            mask2 = create_target_mask(test_filt)
            if total==0:
                success=0
                ratio=0
            else:
                success = len(test_filt[mask2])
                ratio = success / total
#             print(ratio,success,total,row['cutoff'],k)
            roll_mean=test_filt['roll_actual'].mean()
            sim_cutoff_df.loc[len(sim_cutoff_df.index)] = [k,ratio, total, idx, np.nan,roll_mean]

    for group_no,group in enumerate(hash_groups):
        start =group[0]
        end = group[-1]
        mask = (sim_cutoff_df['hash_no']>=start) & (sim_cutoff_df['hash_no']<=end) 
        sim_cutoff_df.loc[mask,'hash_group']=group_no
        
    test_probs['roll_actual']=test_probs['roll_actual_init'].copy()
    
    return sim_cutoff_df,test_probs
        



In [458]:
def get_hash_prob_top_data(test,xgb_models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = None,is_min_total=True,
                           total_cri=12,top_n=10):

    if sim_cutoff_df is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test,xgb_models,features_test,
                                                      test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count)
    else:
        test_probs = generate_probs(True,xgb_models,test,features_test)
        
    print(len(sim_cutoff_df))
#     print(sim_cutoff_df.head())
    #filter only high total records
    if is_min_total:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']>=total_cri]
    else:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']<=total_cri]
        if total_cri!=1:
            sim_cutoff_df_filt = sim_cutoff_df_filt[sim_cutoff_df_filt['total']>1]
    print(len(sim_cutoff_df_filt))
    print(sim_cutoff_df_filt.dtypes)
    sim_cutoff_df_filt['total']=  sim_cutoff_df_filt['total'].astype('float')
    sim_summary  = sim_cutoff_df_filt.groupby(['k']).agg( total =('total','mean'),
                                                        roll_mean=('roll_mean','mean'),
                                                         ratio_mean=('ratio', 'mean'), 
                                                          ratio_std=('ratio', 'std')).reset_index()
#     print(sim_summary.head(20))
#     print(sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=False))
    
    top_mean_data = sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=[False,True])[:top_n].reset_index(drop=True)
    print('--- Top 5 data ---')
    print(top_mean_data[:5])
    print('--- Top 6 to 10 data ---')
    print(top_mean_data[5:])
#     top_std_row = top_mean_data.sort_values('ratio_std',ascending=True).iloc[0]
#     print('--- Selected K ---')
#     print(top_std_row)
#     k_sel = int(top_std_row['k'])

#     k_sel = int(top_mean_data[0:1]['k'])
#     row = test_hash_cutoff_df.iloc[k_sel]
#     #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
#     mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
#     test_sel = test_probs[mask]
#     print('Test Size:',len(test_sel))
    
    return top_mean_data


In [459]:
# k_sel,test_sel = get_test_sel_hash_prob(test,xgb_models,features_test,df,
#                           min_total=12, top_n=10)
# print(k_sel)

In [460]:
def create_cutoff_pos_df(test,models,features_test,test_hash_cutoff_df,
                         imp_feats_count_list,gen_probs=True,is_compute_matches=True,
                   test_ratio_limit=0.12,sim_cutoff_dfs=None,
                         low_high_record_condns=[(True,12,10),(False,10,10),(False,2,10)]):

    cutoff_pos_df =pd.DataFrame(columns = ['imp_feats_count','condn_max','condn_val',
                                           'k_top_1','k_top_2','k_top_3','k_top_4','k_top_5',
                                          'mean_top_1', 'mean_top_2','mean_top_3','mean_top_4',
                                          'mean_top_5','match_pos_1',
                                          'match_total_top_5','match_total_top_10','match_ks_top_5'])

    
    max_good_k= test_hash_cutoff_df[test_hash_cutoff_df['ratio']<test_ratio_limit].index[0]-1
    print(f'{max_good_k=}')
    is_imp_feats = True 
    gen_sim_cutoff=False
    if sim_cutoff_dfs is None:
        gen_sim_cutoff=True
        sim_cutoff_dfs=[]
    elif gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test) 
    else:
        test_probs = test
    for i,imp_feats_count in enumerate(imp_feats_count_list):
        print()
        print(f'********************** Feature count {imp_feats_count} *****************')
        if gen_sim_cutoff:
            sim_cutoff_df,test_probs= get_sim_cutoff_data(test,models,features_test,test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count,gen_probs)
            sim_cutoff_dfs.append(sim_cutoff_df)
        else:
            sim_cutoff_df=sim_cutoff_dfs[i]
        
        for condn in low_high_record_condns:
            print(f'******* Condition {condn} for feature count {imp_feats_count} *****************')
            top_mean_data = get_hash_prob_top_data(test,models,features_test,
                                       test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                                       sim_cutoff_df = sim_cutoff_df,
                                        is_min_total=condn[0], total_cri=condn[1],
                                        top_n=condn[2])
            
#             print(top_mean_data)
            top_5_mean_data = top_mean_data[:5]
            k_tops=list(top_5_mean_data['k'].values)
            mean_tops=list(top_5_mean_data['ratio_mean'].values)
            
            match_pos_1=np.nan
            match_total_top_5=np.nan
            match_total_top_10=np.nan
            match_ks_top_5=np.nan
            
            if is_compute_matches:
                
                top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='average',
                                                                  ascending=False)
                top_5_mean_data = top_mean_data[:5]
               
                matches_top_5=top_5_mean_data[top_5_mean_data['k']<=max_good_k]
                matches_top_5=matches_top_5[matches_top_5['ratio_mean']>0]
                match_total_top_5=len(matches_top_5)

                matches_top=top_mean_data[top_mean_data['k']<=max_good_k]
                matches_top=matches_top[matches_top['ratio_mean']>0]
                match_total_top_10 = len(matches_top)

                if match_total_top_5==0:
                    match_ks_top_5=[]
                    if match_total_top_10==0:
                        match_pos_1=11
                    else:
                        match_pos_1=matches_top.iloc[0]['rank']
                else:
    #                 match_pos_1=matches_top_5.index[0]+1
                    match_pos_1=matches_top_5.iloc[0]['rank']
                    match_ks_top_5 = list(matches_top_5['k'].values)
 
            
            result = [imp_feats_count,condn[0],condn[1]]+k_tops+mean_tops \
                    +[match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5]
        
#             print(cutoff_pos_df.columns)
#             print(f'{len(cutoff_pos_df.columns),len(result),result}')
            next_idx = len(cutoff_pos_df.index)
            cutoff_pos_df.loc[next_idx] = result
        
            if is_compute_matches:
                match_cols = [col for col in cutoff_pos_df if col.startswith('match')]
                print(cutoff_pos_df.loc[next_idx][match_cols])
        
    return cutoff_pos_df,sim_cutoff_dfs,test_probs


In [461]:
import math

def get_test_sel_cutoff_pos(test_probs,models,features_test,test_hash_cutoff_df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                           sim_cutoff_dfs=None,imp_feats_count_list=None):
    cutoff_pos_stat = cutoff_pos_df_hash.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Overall Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_df_hash_filt=cutoff_pos_df_hash[cutoff_pos_df_hash['condn_max']==False]
    cutoff_pos_stat = cutoff_pos_df_hash_filt.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Low Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_best= cutoff_pos_stat.sort_values(['match_pos_1','match_total_top_5'],
                                                ascending=[True,False])\
                                    .reset_index().iloc[0]
    print('*** Best cutoff position in hash **** ')
    print(cutoff_pos_best)
    predicted_raw_pos=cutoff_pos_best['match_pos_1']
    predicted_max_pos = math.ceil(predicted_raw_pos)
    
    if predicted_raw_pos==predicted_max_pos:
        predicted_max_pos+=1
    
    is_imp_feats=True
    imp_feats_count=cutoff_pos_best['imp_feats_count']
    is_min_total=False
    
    if sim_cutoff_dfs is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test_probs,models,features_test,test_hash_cutoff_df,
                                                 is_imp_feats,imp_feats_count,gen_probs=False)
    else:
        #to use existing buffer, read the matching sim cutoff df from the list of buffer
        idx = imp_feats_count_list.index(imp_feats_count)
        sim_cutoff_df=sim_cutoff_dfs[idx]

    top_mean_data = get_hash_prob_top_data(test_probs,models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = sim_cutoff_df,
                            is_min_total=is_min_total, total_cri=total_cri,
                            top_n=top_n)
    
    top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='min',ascending=False)
    mask = (top_mean_data['rank']<=predicted_max_pos) & (top_mean_data['ratio_mean']>0)
    top_mean_sel = top_mean_data[mask]
    k_sel_list = list(top_mean_sel['k'].values)
    print(top_mean_data)
    print(top_mean_sel)
    print(f'{k_sel_list=}')
    

#     mask = cutoff_pos_df_actual['imp_feats_count']==cutoff_pos_best['cutoff_pos_best']
#     mask2 = cutoff_pos_df_actual['match_pos_1']<=predicted_max_pos
#     k_sel = cutoff_pos_df_actual.loc[mask & mask2, 'k' ]
    
    test_sel  = None
    for k_sel in k_sel_list:
        row = test_hash_cutoff_df.loc[k_sel]
    #     print(row)
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        cur_test_sel = test_probs[mask]
        cur_test_sel['k']=k_sel
        if test_sel is None:
            test_sel = cur_test_sel
        else:
            test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
        print('Test Size:',len(test_sel))

    return k_sel_list,test_sel,top_mean_sel



In [462]:
# k_sel_list=[218,198,201,190,184]
# print(df[df.index.isin(k_sel_list)])
# test_temp=None
# for k_sel in k_sel_list:
#     row = df.loc[k_sel]
# #     print(row)
#     mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
#     cur_test_sel = test[mask]
#     cur_test_sel['k']=k_sel
#     if test_temp is None:
#         test_temp = cur_test_sel
#     else:
#         test_temp = pd.concat([test_temp,cur_test_sel],axis=0)

# cols = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:20]        
# # test_temp
# test_temp['feats_mean']=test_temp[cols].mean(axis=1)
# test_temp['feats_std']=test_temp[cols].std(axis=1)
# test_temp['feats_count']=(test_temp[cols]<1000).sum(axis=1)
# test_temp[['k','roll_actual','feats_mean','feats_std','feats_count']+cols]

In [437]:
# test_orig= test.copy()
# test_hash_orig = test_hash.copy()

In [463]:
count_all_k = len(all_k)
if count_all_k<10:
    print(f'PROJECT WARNING: number of target records is {count_all_k} which is less than 10 ')
count_top_k = len(top_k)    
if count_top_k<2:
    print(f'PROJECT WARNING: number of probable success is {count_top_k} which is less than 2')  
hash_success = len(df[(df['total']==2) & (df['ratio']>0)])    
if hash_success<2:
    print(f'PROJECT WARNING: number of hash success is {hash_success} which is less than 2')

In [464]:
print(len(test))
mask = test.duplicated(subset='client_seed')
print(len(test[mask]))
print(test[mask]['client_seed'].nunique())

print(len(test_hash))
mask = test_hash.duplicated(subset='client_seed')
print(len(test_hash[mask]))
print(test_hash[mask]['client_seed'].nunique())

8014
0
0
8103
0
0


In [440]:
# test=test.drop_duplicates(subset='client_seed')
# test_hash=test_hash.drop_duplicates(subset='client_seed')

In [441]:
# %%time
# sim_cutoff_dfs_temp=None
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# cutoff_pos_df_temp,sim_cutoff_dfs_temp,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin([2,22,70,100,178,200,486,
#                                                     488,525,604,672,681,747,
#                                                    812,1212,1392,1536])],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_temp,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_temp

In [442]:
# %%time
# imp_feats_count_list = [30,40]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[df['total']<=10],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,10,20),(False,2,20)])
# cutoff_pos_df_hash

In [443]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [444]:
# print(len(df))
# df_filt = df[(df['total']>=50)]
# print(len(df_filt))
# # df_filt[df_filt['ratio']==0.12]

In [465]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
sim_cutoff_dfs=None
cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
                    features_test,df[(df['total']==1)],
                    imp_feats_count_list,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
                    low_high_record_condns=[(False,1,20)])
cutoff_pos_df_hash

max_good_k=381

********************** Feature count 2 *****************
Generate probs completed
[['roll_37', 'roll_45_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 2 *****************
150
150
k              object
ratio         float64
total          object
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total  roll_mean  ratio_mean  ratio_std
0  491    1.0     1184.5         0.5   0.707107
1  663    1.0     1187.5         0.5   0.707107
2  692    1.0     1217.0         0.5   0.707107
3  552    1.0     1301.0         0.5   0.707107
4  517    1.0     1478.0         0.5   0.707107
--- Top 6 to 10 data ---
      k  total  roll_mean  ratio_mean  ratio_std
5   480    1.0     1483.0         0.5   0.707107
6   587    1.0     1702.0         0.5   0.707107
7   666    1.0     1860.0         0.5   0.707107
8   594    1.0     2181.5         0.5   0.707107
9     2    1.0     3053.5         0.5   0.7071

Generate probs completed
[['roll_37', 'roll_45_nonce', 'roll_mean_50_60', 'roll_54_nonce', 'roll_53_nonce', 'roll_std_50_60_nonce', 'roll_57_nonce', 'roll_33_nonce', 'roll_mean_1_60_nonce', 'roll_28_nonce', 'roll_56_nonce', 'roll_56', 'roll_mean_50_60_nonce', 'roll_mean_25_50', 'roll_mean_1_60', 'roll_46', 'roll_37_nonce', 'roll_48_nonce', 'roll_51_nonce', 'roll_2_nonce', 'roll_30_nonce', 'roll_std_nonce', 'roll_9_nonce', 'roll_50', 'roll_1_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 25 *****************
1875
1875
k              object
ratio         float64
total          object
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  641    1.0  4158.533726        0.24   0.435890
1  517    1.0  3484.024349        0.20   0.408248
2  666    1.0  3745.875462        0.20   0.408248
3  524    1.0  4375.337274        0.20   0.408248
4  687    1.0  4874.578345

Generate probs completed
[['roll_37', 'roll_45_nonce', 'roll_mean_50_60', 'roll_54_nonce', 'roll_53_nonce', 'roll_std_50_60_nonce', 'roll_57_nonce', 'roll_33_nonce', 'roll_mean_1_60_nonce', 'roll_28_nonce', 'roll_56_nonce', 'roll_56', 'roll_mean_50_60_nonce', 'roll_mean_25_50', 'roll_mean_1_60', 'roll_46', 'roll_37_nonce', 'roll_48_nonce', 'roll_51_nonce', 'roll_2_nonce', 'roll_30_nonce', 'roll_std_nonce', 'roll_9_nonce', 'roll_50', 'roll_1_nonce', 'roll_mean_25_50_nonce', 'roll_50_nonce', 'roll_39_nonce', 'roll_24_nonce', 'roll_mean_nonce', 'roll_std_25_50_nonce', 'roll_58', 'roll_49_nonce', 'roll_17_nonce', 'roll_57', 'roll_54', 'roll_21_nonce', 'roll_55_nonce', 'roll_58_nonce', 'roll_31_nonce', 'roll_23_nonce', 'roll_44', 'roll_18_nonce', 'roll_std_1_60_nonce', 'roll_34_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 45 *****************
3375
3375
k              object
ratio         float64
total          object
hash_no        object
hash_group    

******* Condition (False, 1, 20) for feature count 60 *****************
4500
4500
k              object
ratio         float64
total          object
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  524    1.0  4316.110957    0.200000   0.403376
1  666    1.0  3951.504739    0.183333   0.390205
2  517    1.0  3778.861816    0.166667   0.375823
3  578    1.0  3956.897937    0.166667   0.375823
4    3    1.0  5223.925623    0.133333   0.342803
--- Top 6 to 10 data ---
      k  total    roll_mean  ratio_mean  ratio_std
5   691    1.0  4318.216330    0.116667   0.323732
6   446    1.0  4354.913577    0.116667   0.323732
7   649    1.0  4394.856416    0.116667   0.323732
8   552    1.0  4400.310380    0.116667   0.323732
9   563    1.0  4480.462907    0.116667   0.323732
10  531    1.0  4612.676997    0.116667   0.323732
11  491    1.0  4659.857319    0.116667   0.323732
12  687    1.0  4673

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,491,663,692,552,517,0.500000,0.500000,0.500000,0.500000,0.500000,9.5,0,1,[]
1,5,False,1,641,517,2,552,666,0.600000,0.400000,0.400000,0.200000,0.200000,2.5,1,1,[2]
2,10,False,1,641,517,663,552,531,0.300000,0.200000,0.200000,0.200000,0.200000,7.5,0,2,[]
3,15,False,1,641,524,531,517,663,0.266667,0.200000,0.133333,0.133333,0.133333,10.5,0,3,[]
4,20,False,1,517,641,531,491,692,0.250000,0.250000,0.200000,0.200000,0.200000,4.5,0,2,[]
5,25,False,1,641,517,666,524,687,0.240000,0.200000,0.200000,0.200000,0.200000,4.0,0,2,[]
6,30,False,1,517,666,524,641,691,0.266667,0.200000,0.200000,0.200000,0.166667,7.0,0,1,[]
7,35,False,1,517,666,524,641,687,0.228571,0.200000,0.200000,0.171429,0.171429,8.0,0,1,[]
8,40,False,1,517,524,666,687,531,0.225000,0.200000,0.175000,0.175000,0.150000,13.0,0,1,[]
9,45,False,1,517,524,666,687,578,0.200000,0.200000,0.177778,0.155556,0.133333,11.0,0,0,[]


In [466]:
%%time
imp_feats_count_list = [65,70]
sim_cutoff_dfs=None
cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
                    features_test,df[(df['total']==1)],
                    imp_feats_count_list,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
                    low_high_record_condns=[(False,1,20)])
cutoff_pos_df_hash

max_good_k=381

********************** Feature count 65 *****************
Generate probs completed
[['roll_37', 'roll_45_nonce', 'roll_mean_50_60', 'roll_54_nonce', 'roll_53_nonce', 'roll_std_50_60_nonce', 'roll_57_nonce', 'roll_33_nonce', 'roll_mean_1_60_nonce', 'roll_28_nonce', 'roll_56_nonce', 'roll_56', 'roll_mean_50_60_nonce', 'roll_mean_25_50', 'roll_mean_1_60', 'roll_46', 'roll_37_nonce', 'roll_48_nonce', 'roll_51_nonce', 'roll_2_nonce', 'roll_30_nonce', 'roll_std_nonce', 'roll_9_nonce', 'roll_50', 'roll_1_nonce', 'roll_mean_25_50_nonce', 'roll_50_nonce', 'roll_39_nonce', 'roll_24_nonce', 'roll_mean_nonce', 'roll_std_25_50_nonce', 'roll_58', 'roll_49_nonce', 'roll_17_nonce', 'roll_57', 'roll_54', 'roll_21_nonce', 'roll_55_nonce', 'roll_58_nonce', 'roll_31_nonce', 'roll_23_nonce', 'roll_44', 'roll_18_nonce', 'roll_std_1_60_nonce', 'roll_34_nonce', 'roll_std_1_60', 'roll_51', 'roll_53', 'roll_46_nonce', 'roll_52_nonce', 'roll_std_50_60', 'roll_36_nonce', 'roll_std_25_50', 'roll_32

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,65,False,1,524,666,517,578,446,0.2,0.184615,0.169231,0.153846,0.123077,7.5,0,1,[]
1,70,False,1,524,666,517,578,592,0.2,0.171429,0.157143,0.142857,0.142857,13.0,0,1,[]


In [446]:
# %%time
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs=None
# min_normal_count=50
# max_low_count = 5
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']<=max_low_count) | (df['total']>=min_normal_count)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(True,min_normal_count,10),
#                                             (False,max_low_count,20),(False,2,20)])
# cutoff_pos_df_hash

In [447]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [448]:
print(all_k)
print(trans_k)

[3, 4, 6, 10, 11, 12, 13, 14, 15, 26, 65, 82, 91, 320, 327, 343, 344, 351, 365, 389, 411, 426, 428, 431, 432, 433, 435, 440, 442, 443, 444, 447, 448, 450, 455, 460, 462, 463, 468, 469, 479, 486, 488, 489, 492, 493, 496, 500, 502, 508, 510, 515, 517, 520, 521, 530, 536, 541, 542, 544, 547, 550, 553, 556, 560, 561, 566, 578, 644, 653, 654, 663, 671, 681, 690, 691, 695]
[ 26  65  91 320 365 389 389 411 578 644]


In [467]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
sim_cutoff_dfs_actual=None
cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
                    features_test,df[df.index.isin(all_k)],
                    imp_feats_count_list,is_compute_matches=False,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                    low_high_record_condns=[(False,1,20)])
cutoff_pos_df_actual

max_good_k=319

********************** Feature count 2 *****************
Generate probs completed
[['roll_37', 'roll_45_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 2 *****************
154
154
k              object
ratio         float64
total          object
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total  roll_mean  ratio_mean  ratio_std
0  671    1.0      515.5         1.0   0.000000
1  486    1.0      551.0         1.0   0.000000
2   26    1.0      859.0         0.5   0.707107
3  442    1.0     1915.0         0.5   0.707107
4   15    1.0     2056.5         0.5   0.707107
--- Top 6 to 10 data ---
      k  total  roll_mean  ratio_mean  ratio_std
5   443    1.0     2386.5         0.5   0.707107
6   530    1.0     2512.5         0.5   0.707107
7   460    1.0     2518.5         0.5   0.707107
8   432    1.0     2663.5         0.5   0.707107
9    14    1.0     3087.0         0.5   0.7071

Generate probs completed
[['roll_37', 'roll_45_nonce', 'roll_mean_50_60', 'roll_54_nonce', 'roll_53_nonce', 'roll_std_50_60_nonce', 'roll_57_nonce', 'roll_33_nonce', 'roll_mean_1_60_nonce', 'roll_28_nonce', 'roll_56_nonce', 'roll_56', 'roll_mean_50_60_nonce', 'roll_mean_25_50', 'roll_mean_1_60', 'roll_46', 'roll_37_nonce', 'roll_48_nonce', 'roll_51_nonce', 'roll_2_nonce', 'roll_30_nonce', 'roll_std_nonce', 'roll_9_nonce', 'roll_50', 'roll_1_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 25 *****************
1925
1925
k              object
ratio         float64
total          object
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  432    1.0  3343.477952        0.24   0.435890
1  515    1.0  4310.297165        0.24   0.435890
2  520    1.0  4437.782246        0.24   0.435890
3  463    1.0  4174.990472        0.16   0.374166
4  500    1.0  4222.077278

Generate probs completed
[['roll_37', 'roll_45_nonce', 'roll_mean_50_60', 'roll_54_nonce', 'roll_53_nonce', 'roll_std_50_60_nonce', 'roll_57_nonce', 'roll_33_nonce', 'roll_mean_1_60_nonce', 'roll_28_nonce', 'roll_56_nonce', 'roll_56', 'roll_mean_50_60_nonce', 'roll_mean_25_50', 'roll_mean_1_60', 'roll_46', 'roll_37_nonce', 'roll_48_nonce', 'roll_51_nonce', 'roll_2_nonce', 'roll_30_nonce', 'roll_std_nonce', 'roll_9_nonce', 'roll_50', 'roll_1_nonce', 'roll_mean_25_50_nonce', 'roll_50_nonce', 'roll_39_nonce', 'roll_24_nonce', 'roll_mean_nonce', 'roll_std_25_50_nonce', 'roll_58', 'roll_49_nonce', 'roll_17_nonce', 'roll_57', 'roll_54', 'roll_21_nonce', 'roll_55_nonce', 'roll_58_nonce', 'roll_31_nonce', 'roll_23_nonce', 'roll_44', 'roll_18_nonce', 'roll_std_1_60_nonce', 'roll_34_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 45 *****************
3465
3465
k              object
ratio         float64
total          object
hash_no        object
hash_group    

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,671,486,26,442,15,1.000000,1.000000,0.500000,0.500000,0.500000,NaN,NaN,NaN,NaN
1,5,False,1,553,15,432,671,14,0.400000,0.400000,0.400000,0.400000,0.400000,NaN,NaN,NaN,NaN
2,10,False,1,520,15,432,440,671,0.300000,0.200000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN
3,15,False,1,520,440,500,14,496,0.266667,0.200000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN
4,20,False,1,520,432,515,15,440,0.300000,0.200000,0.200000,0.150000,0.150000,NaN,NaN,NaN,NaN
5,25,False,1,432,515,520,463,500,0.240000,0.240000,0.240000,0.160000,0.160000,NaN,NaN,NaN,NaN
6,30,False,1,432,520,455,515,463,0.200000,0.200000,0.200000,0.200000,0.166667,NaN,NaN,NaN,NaN
7,35,False,1,455,432,460,508,520,0.228571,0.171429,0.171429,0.171429,0.171429,NaN,NaN,NaN,NaN
8,40,False,1,455,460,508,515,520,0.225000,0.200000,0.175000,0.175000,0.175000,NaN,NaN,NaN,NaN
9,45,False,1,455,460,508,515,520,0.200000,0.200000,0.177778,0.177778,0.177778,NaN,NaN,NaN,NaN


In [156]:
temp=df.loc[[243,181,220,226,245,10,4]]
temp['diff']=(df.loc[214,'cutoff']-temp['cutoff']).abs()
temp.sort_values('diff')

,cutoff,cutoff_2,ratio,total,diff
220,0.351447,0.352415,0.000000,2,0.028072
10,0.408559,0.409527,0.192308,26,0.029040
245,0.346607,0.347575,0.000000,2,0.032912
243,0.344671,0.345639,0.000000,5,0.034848
226,0.328215,0.329183,0.000000,0,0.051304
4,0.464704,0.465672,0.240964,83,0.085184
181,0.284655,0.285623,0.000000,0,0.094864


In [157]:
# %%time
# # imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs_actual=None
# cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin(trans_k)],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_actual

In [158]:
# k_sel,test_sel,cutoff_pos_best =get_test_sel_cutoff_pos(test_probs,cutoff_pos_df_hash,cutoff_pos_df_actual)
# imp_feats_count_list = [2,5,10,15,20,25]

k_sel,test_sel,top_mean_best =get_test_sel_cutoff_pos(test_probs,xgb_models,features_test,
                                                      df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                            sim_cutoff_dfs=sim_cutoff_dfs_actual,
                            imp_feats_count_list=imp_feats_count_list)
print(f'{k_sel=}')
print('cutoff best')
print(top_mean_best)
print(' ** selected test **')
test_sel[['k','client_seed','probs']]

 ........... Overall Positions of Features .........
   imp_feats_count  match_pos_1  match_total_top_5  match_total_top_10
0                2     8.000000           0.666667            1.000000
1                5     5.333333           1.333333            1.666667
2               10     7.166667           1.000000            1.666667
3               15     8.166667           1.000000            1.666667
4               20     7.333333           0.666667            2.000000
5               25     8.000000           0.666667            1.333333
6               30     8.333333           0.333333            1.000000
7               40    11.000000           0.333333            1.000000
 ........... Low Positions of Features .........
   imp_feats_count  match_pos_1  match_total_top_5  match_total_top_10
0                2        11.00                0.0                 0.0
1                5         7.00                0.5                 0.5
2               10         9.75               

,k,client_seed,probs
1640,189,woxpwoxpwoxpwoxpwoxpeb099c16581e2f1eb598326081e5c1232b51fe6f33091d429b5a5df7065980c9,0.322236
2477,189,woxpwoxpwoxpwoxpwoxp8857f5cc477def19a90090599758fb715f602bf79525d143de0e8e14996c0008,0.321619
3891,222,woxpwoxpwoxpwoxpwoxp957dbb2c6780b08b9cbd5c5c171bbfe28a23914406b9b5cf8da232635f1e6fa1,0.350051
5044,222,woxpwoxpwoxpwoxpwoxp5b56076a42cd0f097c7391ccc0aba2cb6f4fabdbaa4e946015ee20961732259f,0.350428
4154,229,woxpwoxpwoxpwoxpwoxp2d55940bdd307fe9432ce8e5fc422c22704b3c1bc2cc1f5e53d891be95d1708f,0.331125
7326,229,woxpwoxpwoxpwoxpwoxp90643b7aee4cbf42ebf9f383fd48c51ffccd4ca4e31738ef3aa13d9ce33c9ed6,0.331267
1145,241,woxpwoxpwoxpwoxpwoxp5afa5df5cad54ce8a550d8808faf3f992a16ecc30f40c72cf4a8c9a456475f0a,0.343361
3046,241,woxpwoxpwoxpwoxpwoxp1cd2c38f9bcfafaab285b27412e43399bbb1b6fc72854a4beb83215421bc6812,0.343157


In [468]:
test_sel=None
for k_sel in [455,508,432,460,469,515]:
    row = df.iloc[k_sel]
#     print(row)
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    cur_test_sel = test_probs[mask]
    cur_test_sel['k']=k_sel
    if test_sel is None:
        test_sel = cur_test_sel
    else:
        test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
    print('Test Size:',len(test_sel))
test_sel

Test Size: 1
Test Size: 2
Test Size: 3
Test Size: 4
Test Size: 5
Test Size: 6


,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [172]:
imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:10] 
test_sel[['k']+imp_feats]

,k,roll_mean_1_60,roll_42_nonce,roll_60,roll_19_nonce,roll_mean_nonce,roll_mean,roll_mean_25_50_nonce,roll_mean_1_60_nonce,roll_7_nonce,roll_32_nonce
2012,130,4302.066667,9624,9386,5737,5313.9,3959.95,5269.384615,4814.133333,8607,7325
4683,130,5999.000000,1275,9665,7312,6164.5,5416.00,5277.230769,5704.983333,7651,6917
2657,146,4368.300000,9696,9990,7769,5958.4,3656.20,5253.230769,5364.416667,3299,6928
2983,146,4582.866667,7868,9864,6686,5012.5,4039.75,4458.269231,5040.666667,301,4690


In [175]:
sim_hash_group[sim_hash_group['k']==24]

,k,hash_group,ratio_mean,ratio_std
5,24,0.0,0.130841,0.0


In [176]:
sim_cutoff_df[sim_cutoff_df['k']==24]

,k,ratio,total,hash_no,hash_group
48,24,0.130841,214,roll_21_nonce,0.0
49,24,0.130841,214,roll_37_nonce,0.0


In [586]:
def compute_total(data,cutoffs,cols):
    mask = None
    for cutoff,col in zip(cutoffs,cols):
        cur_mask =(data[col]>=cutoff) 
        if mask is None:
            mask = cur_mask
        else:
            mask = mask & cur_mask
    data_filt = data[mask]
    total = len(data_filt)
    return total,data_filt

In [587]:
cols=['roll_20_nonce','roll_14_nonce','roll_3_nonce','roll_16_nonce','roll_11_nonce']
cols=['roll_43_nonce','roll_56_nonce','roll_31_nonce','roll_20_nonce','roll_45_nonce']
cols=['roll_31_nonce','roll_52_nonce','roll_15_nonce','roll_45_nonce','roll_2_nonce']
cols=['roll_31_nonce','roll_52_nonce']
cols=['roll_52_nonce','roll_31_nonce','roll_24_nonce','roll_49_nonce','roll_41_nonce']
cols=['roll_mean_25_50_nonce','roll_52_nonce','roll_31_nonce']
cols=['roll_52_nonce','roll_31_nonce']
cols=['roll_46_nonce','roll_25_nonce']
cols=['roll_42_nonce','roll_37_nonce']

# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce','roll_12_nonce','roll_18_nonce']
# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce']
# cols=['roll_6_nonce','roll_11_nonce','roll_12_nonce','roll_10_nonce',]




In [588]:
def get_cutoff(data,data_hash,col,quantile):
    mask=create_target_mask(data_hash)
#     cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].mean()
#     cutoff = data_hash[mask][col].quantile(0.5)
    cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].quantile(quantile)
    return cutoff



best_ratio =0
best_success = 0
best_total = 0
best_cutoffs = []
best_quantile = 0
best_actual = test_filt
# for quantile in np.linspace(0.01,0.95,50):
for quantile in np.linspace(0.01,0.95,50):
    cutoffs=[]
    for col in cols:
        #use entire hash to determine cutoff
        cutoff = get_cutoff(test,test_hash,col,quantile)
        cutoffs.append(cutoff)
    
#     print(cutoffs)
    total,filt_inter = compute_total(test_filt_hash,cutoffs,cols)
    mask2=create_target_mask(filt_inter)
#     mask2 = (filt_inter['roll_actual']>=9000)
    success = len(filt_inter[mask2])
    
    total_actual,filt_inter_actual = compute_total(test_filt,cutoffs,cols)
    mask2=create_target_mask(filt_inter_actual)

#     mask2 = (filt_inter_actual['roll_actual']>=9000)
    
    if total_actual==0:
        success_actual=0
        ratio_actual=0
    else:
        success_actual = len(filt_inter_actual[mask2])
        ratio_actual = success_actual / total_actual
    print('\nquantile:',quantile)
    print('Actuals:',ratio_actual,success_actual,total_actual)
    
    if total==0:
        ratio=0
    else:
        ratio = success / total
    print('Hash:',ratio,success,total)
#     print(quantile,cutoffs)
    print(total,total_actual)
#     print(total,success)
    if (total==0) or (success==0) : #or (total_actual<1):
        break

#     print(ratio,success,total)
    if ratio >= best_ratio:
        best_ratio = ratio
        best_success = success
        best_total = total
        best_cutoffs = cutoffs
        best_quantile = quantile
        best_actual = filt_inter_actual
print(best_quantile)        
print(best_ratio,best_success,best_total)        
print(best_cutoffs)
# print(test_filt[mask]['roll_actual'].describe())


quantile: 0.01
Actuals: 0.05 1 20
Hash: 0.16666666666666666 2 12
12 20

quantile: 0.02918367346938775
Actuals: 0.05263157894736842 1 19
Hash: 0.1 1 10
10 19

quantile: 0.04836734693877551
Actuals: 0.05263157894736842 1 19
Hash: 0.1 1 10
10 19

quantile: 0.06755102040816326
Actuals: 0.058823529411764705 1 17
Hash: 0.1 1 10
10 17

quantile: 0.086734693877551
Actuals: 0.058823529411764705 1 17
Hash: 0.1111111111111111 1 9
9 17

quantile: 0.10591836734693875
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.12510204081632653
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.14428571428571427
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.16346938775510203
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.1826530612244898
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.20183673469387753
Actuals: 0.0625 1 16
Hash: 0.125 1 8
8 16

quantile: 0.2210204081632653
Actuals: 0.0625 1 16
Hash: 0.125 1 8
8 

In [589]:
mask2=create_target_mask(best_actual)
success = len(best_actual[mask2])
total = len(best_actual)
ratio = success / total
print(ratio,success,total)

0.05 1 20


In [559]:
test_sel = best_actual.copy()

In [261]:


def get_actual_based_data(data,data_hash,data_filt,cols,total_cutoff=3):

    best_filt_inter = data
    best_total= 0
    best_quantile = 0
    for quantile in np.linspace(0.01,0.95,50):
        cutoffs=[]
        for col in cols:
            cutoff = get_cutoff(data,data_hash,col,quantile)
        #     cutoff = get_cutoff(test,test_hash,col)
            cutoffs.append(cutoff)
#         print(quantile,cutoffs)

        total,filt_inter=compute_total(data_filt,cutoffs,cols)
        if total<total_cutoff:
            break
        best_filt_inter = filt_inter
        best_total = total
        best_quantile = quantile
        
    return best_filt_inter,best_total,best_quantile

In [590]:
for cutoff in range(0,11):

    best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,test_filt,cols,
                                                                        total_cutoff=cutoff)
#     print('quantile:',best_quantile)
    mask2=create_target_mask(best_filt_inter)
    # mask2 = (best_filt_inter['roll_actual']>=9000)
    success = len(best_filt_inter[mask2])
    if best_total==0:
        ratio=0
        success=0
    else:
        success = len(best_filt_inter[mask2])
        ratio = success / best_total
    print(cutoff, ratio,success,best_total)
    # print(test_filt[mask]['roll_actual'].describe())

0 0 0 0
1 0.0 0 1
2 0.0 0 2
3 0.0 0 3
4 0.0 0 5
5 0.0 0 5
6 0.0 0 6
7 0.0 0 8
8 0.0 0 8
9 0.0 0 9
10 0.08333333333333333 1 12


In [ ]:
test_sel =best_filt_inter.copy()

In [ ]:
best_filt_inter,best_total,best_quantile=get_actual_based_data(test_filt,test_filt_hash,test_filt,cols,
                                                              total_cutoff=4)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
success = len(best_filt_inter[mask2])
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
# mask_actual = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# idx= test_probs[mask_actual].index
# mask_hash = test_hash_probs.index.isin(idx)
# test_filt_hash_pos = test_hash_probs.loc[mask_hash]

mask_hash = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
idx= test_hash_probs[mask_hash].index
mask_actual = test_probs.index.isin(idx)
test_filt_pos = test_probs.loc[mask_actual]

best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,
                                                               test_filt_pos,cols,
                                                              total_cutoff=37)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
best_filt_inter

In [ ]:
test_sel=best_filt_inter.copy()

In [ ]:
# analysis_df_copy  = analysis_df.copy()

In [166]:
def gen_analysis(nonce,train_manual,test_manual,test,models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",save_files=True,gen_probs=True,
                analysis_folder="analysis"):
    if CONT_ANALYSIS:
        analysis_df=pd.read_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv')
    else:
        analysis_df=None
    print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
    for zero_cutoff in range(0,6):
        analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
                            analysis_df=analysis_df)
    print('Train completed')
    analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
    print('Test completed')   
    
    if save_files:
        joblib.dump(xgb_models,f'data/models/models_{nonce}{file_suffix}.dump')
        print('Models Save completed')   

    features_test = [col for col in features if col not in ['target','probs']]
    
    if gen_probs:
        test=generate_probs(True,models,test,features_test)
    else:
        test['probs'] = np.load(f'data/probs/probs_{nonce}{file_suffix}.npy')

    if save_files:
        np.save(f'data/probs/probs_{nonce}{file_suffix}.npy',test['probs'].values)
    print('Probability Prediction completed')   

    test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
                                                models,
                                               np.mean(np.array(tr_last_cutoffs)),
                                                 np.mean(np.array(val_cutoffs)), 
                                                PATTERN_99,HIGH_ANALYSIS,multi_models=multi_models,
                                                  gen_probs=False)
    print(len(analysis_df))
    analysis_df.to_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv',index=False)  
    print('Analysis Generation completed')  
    return analysis_df,test

In [131]:
test['roll_actual'].head()

0    1119
1    1638
2    2244
3    5933
4    3730
Name: roll_actual, dtype: int64

In [167]:
%%time
analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",
                save_files=True,analysis_folder="analysis_25_to_50_seeds"
#                 gen_probs=False
                    )
analysis_df.head()

Analysis for PATTERN_99:False HIGH_ANALYSIS:False
Train completed
Test completed
Models Save completed
Probability Prediction completed
No Record for zero_col_cutoff:4 digit:7 cutoff:5 after_val:8
No Record for zero_col_cutoff:5 digit:7 cutoff:4 after_val:8
No Record for zero_col_cutoff:5 digit:7 cutoff:5 after_val:8
No Record for zero_col_cutoff:5 digit:6 cutoff:5 after_val:8
Function time:7.605552673339844
972
Analysis Generation completed
CPU times: user 14.8 s, sys: 168 ms, total: 14.9 s
Wall time: 12 s


,nonce,zero_col_cutoff,pattern_99,target_high,digit,cutoff,afterval,tr_ratio,te_ratio,aft_all,aft_1,aft_2,pr_mean,pr_med,pr_1,pr_2,pr_3,pr_4,pr_5,pr_tr_cutoff,pr_val_cutoff,tr_total,tr_success,te_total,te_success,te_pr_1_count,te_pr_2_count,te_pr_3_count,te_pr_4_count,te_pr_5_count,te_mean_count,te_med_count,te_tr_count,te_val_count
0,2649,0,False,False,9,0,25,0.0,0.0,0.0,0.0,0.0,13.636364,15.000000,12.345679,12.345679,12.345679,12.000000,11.111111,11.538462,12.500000,0.0,0.0,0.0,0.0,81.0,81.0,81.0,75.0,18.0,44.0,40.0,78.0,80.0
1,2649,0,False,False,9,0,8,0.0,0.0,0.0,0.0,0.0,8.333333,7.692308,11.538462,11.538462,11.538462,9.090909,0.000000,8.695652,12.000000,0.0,0.0,0.0,0.0,26.0,26.0,26.0,22.0,3.0,12.0,13.0,23.0,25.0
2,2649,0,False,False,9,0,10,0.0,0.0,0.0,0.0,0.0,6.896552,7.407407,10.909091,10.909091,10.909091,10.000000,0.000000,9.615385,11.111111,0.0,0.0,0.0,0.0,55.0,55.0,55.0,50.0,9.0,29.0,27.0,52.0,54.0
3,2649,0,False,False,9,1,25,0.0,0.0,0.0,0.0,0.0,11.363636,11.965812,11.489362,11.489362,11.489362,10.810811,2.380952,10.869565,11.538462,0.0,0.0,0.0,0.0,235.0,235.0,235.0,222.0,42.0,132.0,117.0,230.0,234.0
4,2649,0,False,False,9,1,8,0.0,0.0,0.0,0.0,0.0,17.857143,20.408163,17.171717,17.171717,17.171717,17.391304,0.000000,16.666667,17.346939,0.0,0.0,0.0,0.0,99.0,99.0,99.0,92.0,10.0,56.0,49.0,96.0,98.0


In [168]:
%%time
analysis_df_hash,test_hash = gen_analysis(nonce,train_manual,test_manual_hash,test_hash,
                                xgb_models,tr_last_cutoffs,val_cutoffs,
                                multi_models=True,file_suffix="_hash",
                                save_files=True,
                                         )
analysis_df_hash.head()

Analysis for PATTERN_99:False HIGH_ANALYSIS:False
Train completed
Test completed
Models Save completed
Probability Prediction completed
No Record for zero_col_cutoff:5 digit:7 cutoff:5 after_val:8
Function time:6.502852916717529
972
Analysis Generation completed
CPU times: user 13.8 s, sys: 204 ms, total: 14 s
Wall time: 10.9 s


,nonce,zero_col_cutoff,pattern_99,target_high,digit,cutoff,afterval,tr_ratio,te_ratio,aft_all,aft_1,aft_2,pr_mean,pr_med,pr_1,pr_2,pr_3,pr_4,pr_5,pr_tr_cutoff,pr_val_cutoff,tr_total,tr_success,te_total,te_success,te_pr_1_count,te_pr_2_count,te_pr_3_count,te_pr_4_count,te_pr_5_count,te_mean_count,te_med_count,te_tr_count,te_val_count
0,2649,0,False,False,9,0,25,0.0,0.0,0.0,0.0,0.0,9.615385,8.695652,7.608696,7.608696,7.608696,7.954545,0.000000,7.777778,7.692308,0.0,0.0,0.0,0.0,92.0,92.0,92.0,88.0,20.0,52.0,46.0,90.0,91.0
1,2649,0,False,False,9,0,8,0.0,0.0,0.0,0.0,0.0,11.111111,12.500000,9.375000,9.375000,9.375000,9.375000,0.000000,9.375000,9.375000,0.0,0.0,0.0,0.0,32.0,32.0,32.0,32.0,6.0,18.0,16.0,32.0,32.0
2,2649,0,False,False,9,0,10,0.0,0.0,0.0,0.0,0.0,5.882353,6.666667,6.666667,6.666667,6.666667,6.779661,0.000000,6.779661,6.779661,0.0,0.0,0.0,0.0,60.0,60.0,60.0,59.0,13.0,34.0,30.0,59.0,59.0
3,2649,0,False,False,9,1,25,0.0,0.0,0.0,0.0,0.0,10.156250,7.964602,10.176991,10.176991,10.176991,10.628019,10.526316,10.138249,10.360360,0.0,0.0,0.0,0.0,226.0,226.0,226.0,207.0,38.0,128.0,113.0,217.0,222.0
4,2649,0,False,False,9,1,8,0.0,0.0,0.0,0.0,0.0,5.882353,6.818182,5.617978,5.617978,5.617978,6.097561,14.285714,5.813953,5.747126,0.0,0.0,0.0,0.0,89.0,89.0,89.0,82.0,14.0,51.0,44.0,86.0,87.0


In [ ]:
# %%time
# if CONT_ANALYSIS:
#     analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')
# else:
#     analysis_df=None
# print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
# for zero_cutoff in range(0,6):
#     analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=analysis_df)
# print('Train completed')
# analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=True)
# print('Test completed')

In [ ]:
# %%time

# # model_no=5
# # analysis_df = save_test_ml_probs(analysis_df,test,
# #                                 xgb_models[model_no],np.mean(np.array(tr_last_cutoffs)),
# #                                 np.mean(np.array(val_cutoffs)))


# #Save the Models
# joblib.dump(xgb_models,f'models_{nonce}.dump')

# features_test = [col for col in features if col not in ['target','probs']
# test=generate_probs(True,xgb_models,test,features_test)
# np.save(f'probs/probs_{nonce}.npy',test['probs'].values)
                 
# test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
#                                             xgb_models,
#                                            np.mean(np.array(tr_last_cutoffs)),
#                                              np.mean(np.array(val_cutoffs)), 
#                                             PATTERN_99,HIGH_ANALYSIS,multi_models=True,
#                                               gen_probs=False)
# print(len(analysis_df))
# analysis_df.to_csv(f'data/analysis/analysis_df_{nonce}.csv',index=False)

In [ ]:
# xgb_models= joblib.load(f'data/models/models_{nonce}.dump')
# for model in xgb_models:
#     temp_probs = model.predict_proba(test[0:5][features_test])
#     print(temp_probs)

In [ ]:
# len(analysis_df[(analysis_df['pattern_99']==True) & (analysis_df['target_high']==False)])

In [ ]:
# analysis_df=pd.read_csv(f'data/analysis/analysis_df_2582.csv')

In [169]:
def report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     pattern,bHighCheck,
                    zerocolcutoffs,is_ratio_cri_lessthan=False,
                               is_aftval_opp=False,is_aftval_sum=False):
    mask1 = analysis_df['digit'].isin(digitlist)
    if is_ratio_cri_lessthan:
        mask2= (analysis_df['tr_ratio']<ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']>=ratio_cutoff) | (analysis_df['aft_1']>=ratio_cutoff) | (analysis_df['aft_2']>=ratio_cutoff)) 
    else:
        mask2=  (analysis_df['tr_ratio']>=ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']<ratio_cutoff) | (analysis_df['aft_1']<ratio_cutoff) | (analysis_df['aft_2']<ratio_cutoff)) \
    
        
    mask= mask1 & mask2 \
             & ((analysis_df['aft_all']!=0) & (analysis_df['aft_1']!=0) & (analysis_df['aft_2']!=0)) \
                & (analysis_df['te_total']>=count_cutoff_min) \
                & (analysis_df['te_total']<=count_cutoff_max) \
                & (analysis_df['pattern_99']==pattern) \
                & (analysis_df['target_high']==bHighCheck) \
                & (analysis_df['zero_col_cutoff'].isin(zerocolcutoffs))
        
    if is_aftval_sum:
        mask= mask & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

    filtered = analysis_df[mask]
    
    records  = len(filtered)
    cols = ['tr_ratio','tr_total','te_total']
    
    return filtered

In [ ]:
# %%time

# suffix='_hash'
# digit_list=[1,2,3,4]
# # digit_list=[9,8,7,5]
# zero_col_cutoff_list=[0,1]

# cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#        'pr_tr_cutoff','pr_val_cutoff']
# analysis_list_hash=[]
# mean_list=[]
# model_means=[]
# mask = analysis_df['digit'].isin(digit_list)
# mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
# cutoffs = [(13,20),(13,50),(13,100),(14,20),
#            (14,100),(14,50),(15,20),(15,50),(15,100)]
# cutoffs =[(18,3)]

# # if CONT_ANALYSIS:
# #     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# # else:
# #     analysis_mean_df = None

# analysis_mean_df_hash =None    
# mean_df_hash=None

# for (ratio_cutoff,count_cutoff) in cutoffs:

#     filtered = report_exceptional_teratio(analysis_df_hash,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                          )
#     print(len(filtered))
# #     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
# #                                     xgb_models,
# #                                    np.mean(np.array(tr_last_cutoffs)),
# #                                      np.mean(np.array(val_cutoffs)), 
# #                                   PATTERN_99,HIGH_ANALYSIS,
# #                                   gen_probs=False,
# #                                   multi_models=True )
#     analysis_list_hash.append(filtered)
#     records  = len(filtered)
# #     print('Records:',records)
# #     cur_mean = filtered[cols].mean(axis=0)
#     cur_mean = pd.Series()
#     pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                        'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                       'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                      'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                      'te_ratio':'te_total'}
    
#     for pr_col,pr_count_col in pr_count_dict.items():
#         cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
#     model_mean = cur_mean.mean()
#     te_total_sum = filtered['te_total'].sum(axis=0)
# #     model_mean = filtered[cols].mean(axis=None)
# #     print('Overall mean:',model_mean)
# #     print(cur_mean)

#     cur_mean.loc['overall_mean']=model_mean
#     cur_mean.loc['ratio_cutoff']=ratio_cutoff
#     cur_mean.loc['total_cutoff']=count_cutoff
#     cur_mean.loc['records']=records
#     cur_mean.loc['te_total_sum']=te_total_sum

#     if mean_df_hash is None:
#         mean_df_hash = pd.DataFrame({'temp':cur_mean})    
#     else:
#         mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

# mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
# mean_df_hash['nonce']=nonce
# mean_df_hash['pattern_99']=PATTERN_99
# mean_df_hash['target_high']=HIGH_ANALYSIS
# key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#             'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
#             'overall_mean']
# cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

# mean_df_hash=mean_df_hash[cols_reorder]
# # mean_df
# # temp_df
# if analysis_mean_df_hash is None:
#     analysis_mean_df_hash=mean_df_hash 
# else:
#     analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
# analysis_mean_df_hash

In [ ]:
# %%time

# if not IS_PROD:

#     cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#            'pr_tr_cutoff','pr_val_cutoff']
#     analysis_list=[]
#     mean_list=[]
#     model_means=[]
#     mask = analysis_df['digit'].isin(digit_list)
#     mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
#     #            (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
#     #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
#     # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(18,3)]

# #     if CONT_ANALYSIS:
# #         analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
# #     else:
# #         analysis_mean_df = None

#     analysis_mean_df =None    
#     mean_df=None

#     for (ratio_cutoff,count_cutoff) in cutoffs:

#         filtered = report_exceptional_teratio(analysis_df,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                              )
   
    
# #         filtered = analysis_df[mask1]
#         print(len(filtered))
# #         test_filt_all,filtered = save_test_ml_probs(filtered,test,
# #                                         xgb_models,
# #                                        np.mean(np.array(tr_last_cutoffs)),
# #                                          np.mean(np.array(val_cutoffs)), 
# #                                       PATTERN_99,HIGH_ANALYSIS,
# #                                       gen_probs=False,
# #                                       multi_models=True )
#         analysis_list.append(filtered)
#         records  = len(filtered)
#     #     print('Records:',records)
# #         cur_mean = filtered[cols].mean(axis=0)
#         cur_mean = pd.Series()
#         pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                            'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                           'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                          'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                          'te_ratio':'te_total'}

#         for pr_col,pr_count_col in pr_count_dict.items():
#             cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
#         model_mean = cur_mean.mean()
#         te_total_sum = filtered['te_total'].sum(axis=0)
        
        
#     #     model_mean = filtered[cols].mean(axis=None)
#     #     print('Overall mean:',model_mean)
#     #     print(cur_mean)

#         cur_mean.loc['te_total_sum']=te_total_sum
#         cur_mean.loc['overall_mean']=model_mean
#         cur_mean.loc['ratio_cutoff']=ratio_cutoff
#         cur_mean.loc['total_cutoff']=count_cutoff
#         cur_mean.loc['records']=records

#         if mean_df is None:
#             mean_df = pd.DataFrame({'temp':cur_mean})    
#         else:
#             mean_df=pd.concat([mean_df,cur_mean],axis=1)

#     mean_df=mean_df.transpose().reset_index(drop=True)
#     mean_df['nonce']=nonce
#     mean_df['pattern_99']=PATTERN_99
#     mean_df['target_high']=HIGH_ANALYSIS
#     key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#                 'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
#                 'pr_mean','pr_med','overall_mean']
#     cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

#     mean_df=mean_df[cols_reorder]
#     # mean_df
#     # temp_df
#     if analysis_mean_df is None:
#         analysis_mean_df=mean_df 
#     else:
#         analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
# analysis_mean_df

In [171]:
%%time

suffix='_hash'
digit_list=[1,2,3,4]
digit_list=[9,8,7,5]
# digit_list=[6]
zero_col_cutoff_list=[0,1]
only_better_probs = False
better_prob_cutoff= 11
cutoff_col = 'te_ratio'


cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
       'pr_tr_cutoff','pr_val_cutoff']
analysis_list_hash=[]
mean_list=[]
model_means=[]
mask = analysis_df['digit'].isin(digit_list)
mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
cutoffs = [(13,15),(13,20),(13,50),(13,100),(14,15),(14,20),
           (14,100),(14,50),(15,15),(15,20),(15,50),(15,100),(18,50),(20,50),(25,50),]
# cutoffs =[(14,20)]

# if CONT_ANALYSIS:
#     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# else:
#     analysis_mean_df = None

analysis_mean_df_hash =None    
mean_df_hash=None

for (ratio_cutoff,count_cutoff) in cutoffs:

    mask1= mask & (analysis_df_hash['tr_ratio']>=ratio_cutoff) \
                & (analysis_df_hash['te_total']>=count_cutoff) \
                & (analysis_df_hash['pattern_99']==PATTERN_99) \
                & (analysis_df_hash['target_high']==HIGH_ANALYSIS) \
    
    if only_better_probs:
        mask1 = mask1 & (analysis_df_hash[cutoff_col]>=better_prob_cutoff)

    filtered = analysis_df_hash[mask1]
    print(len(filtered))
#     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
#                                     xgb_models,
#                                    np.mean(np.array(tr_last_cutoffs)),
#                                      np.mean(np.array(val_cutoffs)), 
#                                   PATTERN_99,HIGH_ANALYSIS,
#                                   gen_probs=False,
#                                   multi_models=True )
    analysis_list_hash.append(filtered)
    records  = len(filtered)
#     print('Records:',records)
#     cur_mean = filtered[cols].mean(axis=0)
    cur_mean = pd.Series()
    pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                       'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                      'pr_mean':'te_mean_count','pr_med':'te_med_count',
                     'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                     'te_ratio':'te_total'}
    
    for pr_col,pr_count_col in pr_count_dict.items():
        cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
    model_mean = cur_mean.mean()
    te_total_sum = filtered['te_total'].sum(axis=0)
#     model_mean = filtered[cols].mean(axis=None)
#     print('Overall mean:',model_mean)
#     print(cur_mean)

    cur_mean.loc['overall_mean']=model_mean
    cur_mean.loc['ratio_cutoff']=ratio_cutoff
    cur_mean.loc['total_cutoff']=count_cutoff
    cur_mean.loc['records']=records
    cur_mean.loc['te_total_sum']=te_total_sum

    if mean_df_hash is None:
        mean_df_hash = pd.DataFrame({'temp':cur_mean})    
    else:
        mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
mean_df_hash['nonce']=nonce
mean_df_hash['pattern_99']=PATTERN_99
mean_df_hash['target_high']=HIGH_ANALYSIS
key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
            'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
            'overall_mean']
cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

mean_df_hash=mean_df_hash[cols_reorder]
# mean_df
# temp_df
if analysis_mean_df_hash is None:
    analysis_mean_df_hash=mean_df_hash 
else:
    analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
analysis_mean_df_hash

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
CPU times: user 304 ms, sys: 15 µs, total: 304 ms
Wall time: 303 ms


,nonce,pattern_99,target_high,ratio_cutoff,total_cutoff,records,te_total_sum,te_ratio,pr_tr_cutoff,pr_val_cutoff,pr_mean,pr_med,overall_mean,pr_1,pr_2,pr_3,pr_4,pr_5
0,2649,False,False,13.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2649,False,False,13.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2649,False,False,13.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2649,False,False,13.0,100.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2649,False,False,14.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2649,False,False,14.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2649,False,False,14.0,100.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2649,False,False,14.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2649,False,False,15.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2649,False,False,15.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
analysis_list_hash[0]

In [144]:
print(len(analysis_df_hash))
len(analysis_df_hash[analysis_df_hash['te_ratio']!=0])

972


97

In [145]:
print(len(analysis_df))
len(analysis_df[analysis_df['te_ratio']!=0])

972


95

In [ ]:
%%time

if not False: #IS_PROD:

    cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
           'pr_tr_cutoff','pr_val_cutoff']
    analysis_list=[]
    mean_list=[]
    model_means=[]
    mask = analysis_df['digit'].isin(digit_list)
    mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
    #            (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
    #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
    # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(14,20)]

    if CONT_ANALYSIS:
        analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
    else:
        analysis_mean_df = None

    analysis_mean_df =None    
    mean_df=None

    for i,(ratio_cutoff,count_cutoff) in enumerate(cutoffs):

    #     mask1= mask & (analysis_df['cutoff']==5) \
    #                 & (analysis_df['te_total']>=count_cutoff) \
    #                 & (analysis_df['tr_ratio']>=ratio_cutoff) \
    #                 & (analysis_df['zero_col_cutoff']==3) \
        mask1= mask & (analysis_df['tr_ratio']>=ratio_cutoff) \
                    & (analysis_df['te_total']<=count_cutoff) \
                    & (analysis_df['pattern_99']==PATTERN_99) \
                    & (analysis_df['target_high']==HIGH_ANALYSIS) \
    #                  & (analysis_df['zero_col_cutoff'].isin([0,1])) \
    #                  & (analysis_df['cutoff'].isin([4,5])) \
       #                 & ((analysis_df['aft_all']>13) | (analysis_df['aft_1']>13) | (analysis_df['aft_2']>13)) \
        #                 & ((analysis_df['aft_all']<13) | (analysis_df['aft_1']<13) | (analysis_df['aft_2']<13)) \
        #                  & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

        if only_better_probs:
    #         mask1 = mask1 & (analysis_df.index.isin(analysis_list_hash[i].index))
            filtered=analysis_df.loc[analysis_list_hash[i].index][mask1]
        else:
            filtered = analysis_df[mask1]
            
        print(len(filtered))
        test_filt_all,filtered = save_test_ml_probs(filtered,test,
                                        xgb_models,
                                       np.mean(np.array(tr_last_cutoffs)),
                                         np.mean(np.array(val_cutoffs)), 
                                      PATTERN_99,HIGH_ANALYSIS,
                                      gen_probs=False,
                                      multi_models=True )
        analysis_list.append(filtered)
        records  = len(filtered)
    #     print('Records:',records)
#         cur_mean = filtered[cols].mean(axis=0)
        cur_mean = pd.Series()
        pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                           'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                          'pr_mean':'te_mean_count','pr_med':'te_med_count',
                         'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                         'te_ratio':'te_total'}

        for pr_col,pr_count_col in pr_count_dict.items():
            cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
        model_mean = cur_mean.mean()
        te_total_sum = filtered['te_total'].sum(axis=0)
        
        
    #     model_mean = filtered[cols].mean(axis=None)
    #     print('Overall mean:',model_mean)
    #     print(cur_mean)

        cur_mean.loc['te_total_sum']=te_total_sum
        cur_mean.loc['overall_mean']=model_mean
        cur_mean.loc['ratio_cutoff']=ratio_cutoff
        cur_mean.loc['total_cutoff']=count_cutoff
        cur_mean.loc['records']=records

        if mean_df is None:
            mean_df = pd.DataFrame({'temp':cur_mean})    
        else:
            mean_df=pd.concat([mean_df,cur_mean],axis=1)

    mean_df=mean_df.transpose().reset_index(drop=True)
    mean_df['nonce']=nonce
    mean_df['pattern_99']=PATTERN_99
    mean_df['target_high']=HIGH_ANALYSIS
    key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
                'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
                'pr_mean','pr_med','overall_mean']
    cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

    mean_df=mean_df[cols_reorder]
    # mean_df
    # temp_df
    if analysis_mean_df is None:
        analysis_mean_df=mean_df 
    else:
        analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
    analysis_mean_df

In [ ]:
analysis_mean_df

In [ ]:
analysis_list[0]

In [ ]:
print(test_hash['probs'].describe())
cutoff = test_hash['probs'].quantile(q=0.97)
print(cutoff)
mask = test_hash['probs']>cutoff
total = len(test_hash[mask])
mask2 = test_hash['roll_actual']>=8944
success = len(test_hash[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
print(test['probs'].describe())
cutoff = test['probs'].quantile(q=0.97)
print(cutoff)
mask = test['probs']>cutoff
total = len(test[mask])
mask2 = test['roll_actual']>=8944
success = len(test[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
if not IS_PROD:
    analysis_mean_df.to_csv(f'data/analysis/analysis_mean_df_{nonce}.csv',index=False)

##### Production Code

In [ ]:
# digitlist=[9,8,7,5]
# zerocolcutoffs=list(range(0,6))

# digitlist=[1,2,3,4]
digitlist=[9,8,7,5]
zerocolcutoffs=[4,5]
ratio_cutoff=15
count_cutoff_min=20
count_cutoff_max=10000
analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
anal_size=len(analysis_filt)
test_size=analysis_filt['te_total'].sum()
print(f'Analysis Selection Size: {anal_size} Total Test Size:{test_size}')

In [ ]:
analysis_filt

In [ ]:
#Fetch test selection based on te ratio 

def get_test_from_analysis(analysis_df,test,
              pattern,bHighCheck,
              is_prod=False,):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
        print(f'{filt_size=}')
        mask=create_target_mask(test_filt_df)
        success = len(test_filt_df[mask])
#         success=len(test_filt_df[test_filt_df['roll_actual']>HIGH_TARGET])
        print(f'{success=}')        
        
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue

        if is_prod:
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all

In [ ]:
# temp_sel = save_test_stats(analysis_filt, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
# print(len(temp_sel))

In [ ]:
# %%time
# test_sel = get_test_from_analysis(analysis_filt,test,PATTERN_99,HIGH_ANALYSIS,
#                                   is_prod=True)
# print(len(test_sel))

In [ ]:
%%time
test_sel,ret_analysis_df = save_test_ml_probs(analysis_filt,
                                              test,
                                    xgb_models,
                                   np.mean(np.array(tr_last_cutoffs)),
                                  np.mean(np.array(val_cutoffs)), 
                                  PATTERN_99,HIGH_ANALYSIS,
                                  is_prod=True,
                                 gen_probs=False,
                                  prob_cutoff=0.3,
                                  multi_models=True )
print(f'Selected Test Size:{len(test_sel)}')

In [ ]:
if not IS_PROD:
    success=len(test_sel[test_sel['roll_actual']>=9000])
    print(success)
    print(success / len(test_sel))

In [ ]:
test_sel.head()

In [469]:
# def get_random_client_seed(test):
#     size = len(test)
#     test = test.reset_index(drop=True).sample(n=size).reset_index(drop=True)
#     rand_pos = random.randint(0,size-1)
#     print(f'rand_pos:{rand_pos}')
#     return test.iloc[rand_pos]['client_seed'],test

def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],final_test

In [470]:
client_seed,final_test = get_random_client_seed(test_sel)
print('Client Seed Selected')
print(client_seed)
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')
# final_test.head()

rand_pos:5
Client Seed Selected
woxpwoxpwoxpwoxpwoxpcf0e00e2b60b297d6b6dcb153888a52b070849ac798b0ac10480586d5bdae7eb


In [471]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [147]:
final_test.reset_index()[final_test.reset_index()['index']==6826]

,index,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,...,roll_actual_13,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll_actu

After Actual SEED is known

In [472]:
actual_seed="b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263"
# actual_seed=cur_hash_list[4]
# print(f'{actual_seed=}')

In [473]:
cur_hash_list[25]

'7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191'

In [474]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       final_test['client_seed'],
                                       nonce)
final_test['roll_actual_dummy']=final_test['roll_actual'].copy()
final_test['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [475]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll_actual_30,roll

In [287]:
final_test.to_csv(f'data/final_test_{nonce}_{file_pattern_str}_pattern.csv')

In [162]:
LOW_TARGET

1000

In [159]:
mask = create_target_mask(final_test)
success_pos = final_test[mask].index

if HIGH_ANALYSIS:
    success_pos = final_test[final_test['roll_actual']>=HIGH_TARGET].index
else:
    success_pos = final_test[final_test['roll_actual']<LOW_TARGET].index
print(len(success_pos))
print(f'success_pos:{success_pos}')

# target = set([17, 19, 34, 35, 44, 47])
# matched  = set(list(success_pos)).intersection(target)
# print(matched)

# print(len(matched)/len(success_pos))

1
success_pos:Int64Index([6865], dtype='int64')


In [ ]:
test_sel.head()

In [ ]:
final_test

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)


analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="")
analysis_df.head()

In [ ]:
# ratio_cutoff=18
# count_cutoff_min=3
# # test_sel_options= [([1],[0,1],10000),
# #                     ([2],[0,1],10000),
# #                    ([3],[0,1],10000),
# #                    ([4],[0,1],10000),
# #                     ([5],[0,1],10000),
# # #                     ([6],[0,1],10000),
# #                    ([7],[0,1],10000),
# #                    ([8],[0,1],10000),
# #                    ([9],[0,1],10000),
# #                     ([1],[4,5],10000),
# #                     ([2],[4,5],10000),
# #                    ([3],[4,5],10000),
# #                    ([4],[4,5],10000),
# #                     ([5],[4,5],10000),
# # #                     ([6],[4,5],10000),
# #                    ([7],[4,5],10000),
# #                    ([8],[4,5],10000),
# #                    ([9],[4,5],10000),
# #                   ]

# test_sel_options= [([1,2,3,4],[0,1],10000),
#                     ([9,8,7,5],[0,1],10000),
#                    ([6],[0,1],10000),
#                     ([1,2,3,4],[4,5],10000),
#                     ([9,8,7,5],[4,5],10000),
#                    ([6],[4,5],10000),
#                   ]
# test_sel_options= [([1],[0],10000),]

#                    #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),

# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]


# # test_sel_options= [([9,8,7,6,5,1,2,3,4],[0,1,4,5],10000)]
# # test_sel_options= [([9,8,7,5],[4,5],10000)]
# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]

# anal_sizes =[]
# analysis_filts=[]
# test_sizes=[]
# filt_options=[]
# for option in test_sel_options:
#     digitlist=option[0]
#     zerocolcutoffs=option[1]
#     count_cutoff_max=option[2]
#     analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
#                          ratio_cutoff,count_cutoff_min,count_cutoff_max,
#                          PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
#     anal_size=len(analysis_filt)
#     if anal_size!=0:
#         test_size=analysis_filt['te_total'].sum()
#         print(f'option: {option} Analysis Selection Size: {anal_size} Total Test Size:{test_size}')
#         break
# #         anal_sizes.append(anal_size)
# #         analysis_filts.append(analysis_filt)
# #         filt_options.append(option)
# #         test_sizes.append(test_size)
#     else:
#         print(f'No records for option:{option}')

# # sel_option = np.argmin(np.array(test_sizes))
# # print(f'selected option: {filt_options[sel_option]} \
# #       Analysis Selection Size: {anal_sizes[sel_option]} Total Test Size:{test_sizes[sel_option]}')

# # analysis_filt = analysis_filts[sel_option]

#### END

Finalize Training

In [ ]:
tr_index

In [ ]:
train.index

In [ ]:
# tr_index = train[30 * train_client_size:(30 + 5) * train_client_size].index
tr_index = train[train_client_size:].index
train_ignored = train[0:30*train_client_size]
# tr_index = list(tr_index) + list(train_ignored[train_ignored['target']==1].index)
print(len(tr_index))
xgb_model_final = xgb.XGBClassifier(**params)

X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# fit and apply the transform
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
print(pd.Series(y_tr).value_counts())
xgb_model_final.fit(X_tr,y_tr)

In [ ]:
len(X_tr)

In [ ]:
test_probs = xgb_model_final.predict_proba(test)[:,1]

test_labels = convert_probtolabels(test_probs)
subm=pd.DataFrame()
subm['client_seed']= results_df_test['client_seed']
subm['preds'] = test_labels
subm['prob'] = test_probs
score = accuracy_score(y_test,test_labels)
print(f'test score:{score}')

In [ ]:
mask = (subm['prob']<0.52) & (subm['preds']==1)
print(len(subm[mask]))
score = accuracy_score(y_test[mask],test_labels[mask])
print(f'test score:{score}')